In [1]:
#Packages

import math
import os
import glob
import csv
import shutil

import warnings
warnings.simplefilter('error', RuntimeWarning)

from math import log10, floor

import numpy as np
import numpy.ma as ma
from numpy import arange,array,ones
from numpy import exp, linspace, random

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.mlab as mlab
from matplotlib import cm
from matplotlib.offsetbox import AnchoredText
from matplotlib.colors import LogNorm

from astropy.io import fits
from astropy.modeling import models, fitting
import astropy.io.ascii as a

# from uncertainties import unumpy
# from uncertainties import ufloat

import pylab as plb

from scipy.optimize import curve_fit
from scipy.optimize import leastsq as lmfitter
from scipy import signal
from scipy import asarray as ar,exp
from scipy.stats import chisquare
from scipy.stats import stats
from scipy.integrate import quad
from scipy.stats import linregress
from scipy.stats import skew, kurtosis

import scipy.io
import scipy.integrate as integrate

from IPython.display import display, Math

from itertools import chain

#import sherpa.ui as ui
import pandas as pd
from sklearn.linear_model import LinearRegression

# import marvin
# #from marvin.tools import Maps
# from marvin.utils.general.images import showImage
# from marvin.tools.cube import Cube
# from marvin import config

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)

%matplotlib inline

In [2]:
#Constants (Vacuum Wavelengths Reported)

OVI = 1033.82 
Ly_Alpha = 1215.24  
NV = 1240.81  
OI = 1305.53  
CII = 1335.31  
Si_IV = 1397.61  
Si_IV_O_IV = 1399.8  
CIV = 1549.48  
He_II = 1640.4
OIII = 1665.85 
Al_III = 1857.4 
CIII = 1908.734 
CII = 2326.0 
Ne_IV = 2439.5 
Mg_II = 2799.117 
NeV_3346 = 3346.79 #*
NeV_3426 = 3426.85 
OIII_0 = 3444.052
NI_0 = 3466.497 
He_0 = 3487.727
FeVII_3586 = 3586 #*
OII_1 = 3727.092 ###
OII_2 = 3729.875  
FeVII_3760 = 3760 #*
He_I = 3889.0 
SII = 4072.3 
H = 4102.89 
H = 4341.68  
OIII_1 = 4364.436  
H_Beta = 4862.68  
OIII_2 = 4932.603  
OIII_3 = 4960.295  
OIII_4 = 5008.240  
He_I_1 = 5875
OI_0 = 6046
FeVII_6086 = 6086 #*
OI_1 = 6302.046  
OI_2 = 6365.536  ###
FeX_6374 = 6374 #* 
NI = 6529.03  
NII_1 = 6549.86 
H_Alpha = 6564.614  
NII_2 = 6585.27  
SII = 6718.29  
SII = 6732.67  

#Absorption Lines

K = 3934.777  
H = 3969.588 
G = 4305.61  
Mg = 5176.7  
Na = 5895.6  
CaII_1 = 8500.36  
CaII_2 = 8544.44  
CaII_3 = 8664.52  

#Sky Lines

Sky_1 = 5578.5 
Sky_2 = 5894.6  
Sky_3 = 6301.7  
Sky_4 = 7246.0 

c = 299792 #speed of light km/s 
H_0 = 70 #Hubble constant km/mpc/s

In [2]:
def emission(emline, bounds, emline_name):
    for fitsName in glob.glob('output/*.fits'):

        hdulist = fits.open(fitsName) 
        wavelength_import = hdulist[4].data #Importing Wavelength Values
        flux_import = hdulist[1].data
        ivar = hdulist[2].data

        drpall = fits.open('drpall-v2_5_3.fits') #Opening drpall file
        tbdata = drpall[1].data #Accessing drpall data
        ind = np.where(tbdata['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
        iau = tbdata['nsa_iauname'][ind][0] #Finding SDSS Name
        plateifu1 = tbdata['plateifu'][ind][0]
        sersic_mass = tbdata['nsa_sersic_mass'][ind][0] #Stellar mass from K-correction fit in h-2 solar masses to sersic magnitudes. 

        dapall = fits.open('dapall-v2_5_3-2.3.0.fits') #Opening dapall file
        tbdata_1 = dapall[1].data #Accessing dapall data
        ind_1 = np.where(tbdata_1['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
        velocity_dispersion = tbdata_1['STELLAR_SIGMA_1RE'][ind][0] 
        smbh_mass = (3.1*(velocity_dispersion/200)**4)*(10**8)
        distance = ((tbdata['z'][ind][0])*c)/H_0
        sfr = tbdata_1['SFR_1RE'][ind][0] 

        wavelength = wavelength_import/(tbdata['z'][ind][0] + 1)

        if emline == NeV_3346 and tbdata['z'][ind][0] < 0.0882:
            hdulist.close()
            drpall.close()  
            break

        if emline == NeV_3426 and tbdata['z'][ind][0] < 0.0628:
            hdulist.close()
            drpall.close()  
            break
            
        if emline == FeVII_3586 and tbdata['z'][ind][0] < 0.0156:
            hdulist.close()
            drpall.close()  
            break

        index1,index2,index3 = flux_import.shape
        cp = int(index2/2)
        cp_down = cp-3
        cp_up = cp+3
        pd = int(index3/2)
        p_down = pd-3
        p_up = pd+3

        #Nuclear region flux
        flux_nuclear_dimensions = flux_import[:,cp_down:cp_up,p_down:p_up]
        flux_nuclear_dimensions_ivar = ivar[:,cp_down:cp_up,p_down:p_up]
        flux_nuclear_sum = flux_nuclear_dimensions.sum(axis = (0))
        flux_nuclear_sum_ivar = flux_nuclear_dimensions_ivar.sum(axis = (0))
        #flux = flux_nuclear_dimensions.sum(axis = (1, 2))
        flux_ivar = flux_nuclear_dimensions_ivar.sum(axis = (1, 2))

        #Galaxy flux
        flux_galaxy = flux_import.sum(axis = (0))
        flux = flux_import.sum(axis = (1,2))
        #flux_galaxy_1d = flux_import.sum(axis = (1,2))

        #Defining neighboring wavelengths
        wavelength_absolute = np.abs(wavelength - emline) 
        wavelength_min = np.argmin(wavelength_absolute)

        #FeVII_3076 Neighboring Lines
        wavelength_oii1 = np.abs(wavelength - OII_1) 
        wavelength_min_oii1 = np.argmin(wavelength_oii1)
        wavelength_oii2 = np.abs(wavelength - OII_2) 
        wavelength_min_oii2 = np.argmin(wavelength_oii2)
        wavelength_hei = np.abs(wavelength - He_I) 
        wavelength_min_hei = np.argmin(wavelength_hei)

        #FeVII_6086 Neighboring Lines
        wavelength_na = np.abs(wavelength - Na) 
        wavelength_min_na = np.argmin(wavelength_na) 
        wavelength_oi0 = np.abs(wavelength - OI_0) 
        wavelength_min_oi0 = np.argmin(wavelength_oi0) 
        wavelength_hei1 = np.abs(wavelength - He_I_1) 
        wavelength_min_hei1 = np.argmin(wavelength_hei1) 

        #FeX_6374 Neighboring Lines
        wavelength_oi1 = np.abs(wavelength - OI_1) 
        wavelength_min_oi1 = np.argmin(wavelength_oi1)
        wavelength_oi2 = np.abs(wavelength - OI_2) 
        wavelength_min_oi2 = np.argmin(wavelength_oi2)
        wavelength_ni = np.abs(wavelength - NI) 
        wavelength_min_ni = np.argmin(wavelength_ni)
        wavelength_nii1 = np.abs(wavelength - NII_1) 
        wavelength_min_nii1 = np.argmin(wavelength_nii1)
        wavelength_nii2 = np.abs(wavelength - NII_2) 
        wavelength_min_nii2 = np.argmin(wavelength_nii2)
        wavelength_halpha = np.abs(wavelength - H_Alpha) 
        wavelength_min_halpha = np.argmin(wavelength_halpha)

        #NeV_3346 Neighboring Lines
        wavelength_nevi = np.abs(wavelength - NeV_3426) 
        wavelength_min_nevi = np.argmin(wavelength_nevi)

        #NeV_3426 Neighboring Lines
        wavelength_nev = np.abs(wavelength - NeV_3346) 
        wavelength_min_nev = np.argmin(wavelength_nev)

        #H-Alpha Neighboring Lines
        wavelength_oiii3 = np.abs(wavelength - OIII_3) 
        wavelength_min_oiii3 = np.argmin(wavelength_oiii3)

        #Determining observed (for vmeasured value)
    #         lambda_obs = (tbdata['z'][ind][0] + 1)*emline
    #         wavelength_absolute = np.abs(wavelength - lambda_obs) 
    #         wavelength_min = np.argmin(wavelength_absolute)

        #Spectrum parameters
        x_0, x_1 = wavelength_min - bounds, wavelength_min + bounds #Setting lower bound for linear fit
        if x_0 < 0:
            x_0 = 0
        #Setting upper bound for linear fit
        z_1_2 = x_0 - 10*bounds
        if z_1_2 < 0:
            z_1_2 = 0
        x_1_2 = x_1 + 10*bounds

        wavelength_bounds_total = wavelength[z_1_2:x_1_2] 
        wavelength_total = np.abs(wavelength_bounds_total - emline) 
        wavelength_min_total = np.argmin(wavelength_total)

        wavelength_mask_total = np.ma.masked_array(wavelength_bounds_total,
                                             (wavelength_bounds_total != wavelength_bounds_total[0])& 
                                             (wavelength_bounds_total != wavelength_bounds_total[-1])) 
        wavelength_compressed_total = wavelength_mask_total.compressed() 

        flux_bounds_total = flux[z_1_2:x_1_2] 
        flux_bounds_total_ivar = flux_ivar[z_1_2:x_1_2] 

        flux_mask_total = np.ma.masked_array(flux_bounds_total, (flux_bounds_total != flux_bounds_total[0]) 
                                   & (flux_bounds_total != flux_bounds_total[-1]))
        flux_compressed_total = flux_mask_total.compressed() 

        if flux_compressed_total.shape[0] == 3:
            print ('duplicate')
            flux_1,flux_2 = flux_compressed_total[0],flux_compressed_total[2]
            flux_compressed_total = [flux_1, flux_2]

        if flux_compressed_total[0] == 0:
                hdulist.close()
                drpall.close()  
                print ('Compressed Flux Error')
                break

        else:

            polyfit_total = np.polyfit(wavelength_compressed_total,flux_compressed_total, 1) #Fitting the line to the data
            fit_total = np.poly1d(polyfit_total)
            flux_linear_total = fit_total[1]*wavelength_bounds_total + fit_total[0] #Determining linear fit parameters
            flux_correction_total = flux_bounds_total - flux_linear_total #Correcting for continuum

            for i in flux_correction_total:
                if i < 0:
                    negative = np.argmin(flux_correction_total)
                    flux_correction_total = flux_correction_total + abs(flux_correction_total[negative])
                    break

            q_0, q_1 = wavelength_min_total - bounds, wavelength_min_total + bounds
            if q_0 < 0:
                q_0 = 0
            wavelength_bounds, flux_bounds = wavelength_bounds_total[q_0:q_1], flux_correction_total[q_0:q_1]
            #flux_bounds_error = np.sqrt(1/flux_bounds_total_ivar[q_0:q_1])

            wavelength_em = np.abs(wavelength_bounds - emline) 
            wavelength_bounds_min = np.argmin(wavelength_em)

            wavelength_mask = np.ma.masked_array(wavelength_bounds,
                                             (wavelength_bounds != wavelength_bounds[0])& 
                                             (wavelength_bounds != wavelength_bounds[-1])) 
            wavelength_compressed = wavelength_mask.compressed() 

            flux_mask = np.ma.masked_array(flux_bounds, (flux_bounds != flux_bounds[0]) 
                                   & (flux_bounds != flux_bounds[-1]))
            flux_compressed = flux_mask.compressed() 

            if flux_compressed.shape[0] == 3:
                print ('duplicate')
                flux_compressed = [flux_1, flux_2]

            if flux_compressed[0] == 0:
                hdulist.close()
                drpall.close()  
                break

            else:

                polyfit = np.polyfit(wavelength_compressed,flux_compressed, 1) #Fitting the line to the data
                fit = np.poly1d(polyfit)
                flux_linear = fit[1]*wavelength_bounds + fit[0] #Determining linear fit parameters
                avg_flux = np.average(flux_linear)

            x_1_9 = x_1 + 4*bounds + 5
            flux_sigma_bounds_1 = flux[x_1 + 5: x_1_9]
            if emline == NeV_3426:
                flux_sigma_bounds_1 = flux[x_1_9 + 5: x_1 + 8*bounds + 10]
            f_std = np.std(flux_sigma_bounds_1)
            flux_sigma = 5*(f_std)
            flux_sigma_1 = 1*(f_std)

            if len(fitsName) == 57:
                print ('\n' + iau + ' ('+ fitsName[13:22] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))
            if len(fitsName) == 58:
                print ('\n' + iau + ' (' + fitsName[13:23] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))
            if len(fitsName) == 59:
                print ('\n' + iau + ' (' + fitsName[13:24] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))

            def gaus(x,amp,mu,sigma, m, c):
                return amp*np.exp(-(x-mu)**2/(2*sigma**2)) + m*x + c #Defining Gaussian function

            def double_gaussian(x,amp1,mu1,sigma1,m1,c1,amp2,mu2,sigma2,m2,c2):
                return gaus(x,amp1,mu1,sigma1,m1,c1) + gaus(x,amp2,mu2,sigma2,m2,c2)

            wavelength_emission = np.argmax(flux_bounds)
            amp_ems = flux_bounds[wavelength_emission]
            wavelength_absorption = np.argmin(flux_bounds)
            amp_abs = flux_bounds[wavelength_absorption]
            amp_abs_1 = -(avg_flux - amp_abs)
            sigma_abs_1 = avg_flux - flux_sigma_1
            sigma_abs_5 = avg_flux - flux_sigma
            sigma_ems = avg_flux + flux_sigma

            o1_wavelength_total = np.abs(wavelength_bounds - OI_2) 
            o1_wavelength_min_total = np.argmin(o1_wavelength_total)
            amp_o1 = flux_bounds[o1_wavelength_min_total]
            mu_o1 = wavelength_bounds[o1_wavelength_min_total]
            sigma_guess_o1 = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_o1)**2)/np.sum(flux_bounds))

            mu_abs = wavelength_bounds[wavelength_absorption]
            mu_ems = wavelength_bounds[wavelength_emission]
            mu = np.sum(wavelength_bounds*flux_bounds)/np.sum(flux_bounds)

            sigma_guess = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu)**2)/np.sum(flux_bounds))
            amp = flux_bounds[wavelength_bounds_min_total]

            if amp_abs < sigma_abs_5:
                hdulist.close()
                drpall.close() 
                print ('5 Sigma Absorption Line Error')
                break

            if amp_abs < sigma_abs_1 and amp_ems < sigma_ems:

                try:
                    popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                               p0=[amp_abs_1, mu_abs, sigma_guess, fit[1], 
                                   fit[0]])
                except RuntimeError:
                    hdulist.close()
                    drpall.close() 
                    print ('RuntimeError')
                    break

                print (emline_name + ' Absorption Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))

                if popt[0] < 0:
                    hdulist.close()
                    drpall.close() 
                    print ('Absorption Line Error')
                    break

                else:
                    try:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                   p0=[amp_ems, mu, sigma_guess, fit[1], 
                                       fit[0]])
                    except RuntimeError:
                        hdulist.close()
                        drpall.close() 
                        print ('RuntimeError')
                        break

                    try:
                        perr = np.sqrt(np.diag(pcov))
                    except RuntimeWarning:
                        print ('RuntimeWarning')
                        hdulist.close()
                        drpall.close()
                        break

                    residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata
                    chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                    reduced_chi_squared = chi_squared / (len(popt) - 1)

                    print ('Chi Squared test = ' +  str(chi_squared) + '\n' + 'Reduced Chi Squared = ' 
                           + str(reduced_chi_squared))

                    print (emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))

            else:
                try:
                    popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                               p0=[amp_ems, mu, sigma_guess, fit[1], 
                                   fit[0]])
                except RuntimeError:
                    hdulist.close()
                    drpall.close() 
                    print ('RuntimeError')
                    break

                try:
                    perr = np.sqrt(np.diag(pcov))
                except RuntimeWarning:
                    print ('RuntimeWarning')
                    hdulist.close()
                    drpall.close()

                residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata
                chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                reduced_chi_squared = chi_squared / (len(popt) - 1)

                print ('Single Gaussian Chi Squared = ' +  str(chi_squared) + '\n' + 'Single Gaussian Reduced Chi Squared = ' 
                       + str(reduced_chi_squared))

                print (emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))

                print ('skewness scipy = ' + str(skew(gaus(wavelength_bounds, *popt))))

                I = np.sum(gaus(wavelength_bounds, *popt))
                x_bar = (1/I)*np.sum(wavelength_bounds*gaus(wavelength_bounds, *popt))
                std_1 = np.sqrt((1/I)*np.sum((wavelength_bounds - x_bar)**2*gaus(wavelength_bounds, *popt)))
                S = (1/(I*std_1**3))*np.sum((wavelength_bounds - x_bar)**3*gaus(wavelength_bounds, *popt))
                print ('skewness lit = ' + str(S))
#                     if emline == FeX_6374:    
#                         try:
#                             popt_dg, pcov_dg = curve_fit(double_gaussian,wavelength_bounds, flux_bounds,  
#                                            p0=[amp,mu,sigma_guess,fit[1],fit[0],amp_o1,mu_o1,sigma_guess_o1,fit[1],fit[0]]) 
#                         except RuntimeError:
#                             hdulist.close()
#                             drpall.close() 
#                             print ('RuntimeError')
#                             break

#                         try:
#                             perr_dg = np.sqrt(np.diag(pcov_dg))
#                         except RuntimeWarning:
#                             print ('RuntimeWarning')
#                             hdulist.close()
#                             drpall.close()
#                             break
#                         residuals_dg = flux_bounds - double_gaussian(wavelength_bounds, *popt_dg) #Determines the uncertainty in ydata
#                         chi_squared_dg =  np.sum(((residuals_dg)** 2)/(double_gaussian(wavelength_bounds, *popt_dg)))
#                         reduced_chi_squared_dg = chi_squared_dg / (len(popt_dg) - 1)

#                         print ('Double Gaussian Chi Squared = ' +  str(chi_squared_dg) + '\n' + 'Double Gaussian Reduced Chi Squared = ' 
#                                + str(reduced_chi_squared_dg))

#                         print (emline_name + ' OI Flux (ergs/cm^2/A/spaxel) = ' + str(popt_dg[5]) + '\n'
#                                + emline_name + ' FeX Flux (ergs/cm^2/A/spaxel) = ' + str(popt_dg[0]))

#                         velocity_meas_dg = c*((popt_dg[1] - emline)/emline)
#                         velocity_sys_dg = c*tbdata['z'][ind][0]
#                         velocity_off_dg = velocity_sys_dg - velocity_meas_dg
#                         sigma_dg = c*(abs(popt_dg[2])/emline)
#                         fwhm_dg = sigma_dg*2.355
#                         ip = 262.1

#                         #if sigma_dg > 0 and sigma_dg < 450 and popt_dg[0] > flux_sigma and mu-15 < popt_dg[1] < mu+15 and popt_dg[0] > 3*perr_dg[0]:

#                         newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/5_sigma/%s/'%emline_name + '%s/'%iau
#                         if not os.path.exists(newpath):
#                             os.makedirs(newpath)
#                         f = open(newpath + "dg_data.csv","w+")
#                         f.write('SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + '5 Sigma Threshold (ergs/cm^2/A/spaxel)'
#                                  + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
#                                 + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
#                                 + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
#                                 + 'Gaussian Sigma (A)' + ',' 
#                                 + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM (km/s)' + ',' + 'Measured Velocity (km/s)' 
#                                 + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
#                                  + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)'+
#                                 '\n' + str(iau) + ',' + str(tbdata['z'][ind][0]) + ',' + str(distance) + ',' + str(flux_sigma) + ',' 
#                                 + str(popt_dg[0])+ ',' + str(perr_dg[0]) 
#                                 + ','  + str(popt_dg[1]) + ',' + str(perr_dg[1]) +  ','  + str(popt_dg[2]) + ',' + str(perr_dg[2]) + ',' 
#                                 + str(sigma_dg) + ','  + str(fwhm_dg) + ',' + str(velocity_meas_dg)+ ',' + str(smbh_mass) + ',' + str(sersic_mass) 
#                                 + ','  + str(velocity_dispersion) + ',' + str(sfr) + ',' + str(chi_squared_dg) + ',' + str(reduced_chi_squared_dg) 
#                                 + ',' + str(ip))
#                         f.close()

#                         fig = plt.figure(figsize=(15,10))
#                         ax1 = plt.axes()  
#                         ax2 = plt.axes([0.18, 0.7, 0.15, 0.15])
#                         ax1.plot(wavelength_bounds_total, flux_correction_total, 'b', label = 'Data')

#                         ax2.plot(wavelength_bounds, residuals, 'k')
#                         ax2.set_title('Residuals')
#                         ax2.set_ylabel('Flux')
#                         ax2.set_xlabel(r'Wavelength ($\AA$)')

#                         if len(fitsName) == 57:
#                             ax1.set_title(iau + ' ('+ fitsName[13:22] + ')')
#                         if len(fitsName) == 58:
#                             ax1.set_title(iau + ' ('+ fitsName[13:23] + ')')
#                         if len(fitsName) == 59:
#                             ax1.set_title(iau + ' ('+ fitsName[13:24] + ')')

#                         ax1.set_ylabel('Flux', fontsize = 15)
#                         ax1.set_xlabel(r'Wavelength ($\AA$)', fontsize = 15)
#                         r = wavelength_bounds_total.shape
#                         ax1.set_xlim(wavelength_bounds_total[1],wavelength_bounds_total[r[0] - 1])

#                         marker = np.argmin(flux_correction_total)
#                         marker_min = np.argmax(flux_correction_total)
#                         marker_threshold = -(0.035*flux_correction_total[marker_min])

#                         ax1.text(wavelength[x_1 + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)


#                         ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
#                         ax1.text(wavelength[wavelength_min-3], marker_threshold, 'FeX', fontsize=9)
#                         ax1.plot(wavelength[wavelength_min_oi2],flux_correction_total[marker], marker='|', color = 'purple', linestyle='None')
#                         ax1.text(wavelength[wavelength_min_oi2-2], marker_threshold, 'OI', fontsize=9)
#                         ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
#                         ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
#                         ax1.plot(wavelength[wavelength_min_ni],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
#                         ax1.text(wavelength[wavelength_min_ni-2], marker_threshold, 'NI', fontsize=9)
#                         ax1.plot(wavelength[wavelength_min_nii1],flux_correction_total[marker], marker='|', color = 'magenta', linestyle='None')
#                         ax1.text(wavelength[wavelength_min_nii1-3], marker_threshold, 'NII', fontsize=9)
#                         ax1.plot(wavelength[wavelength_min_nii2],flux_correction_total[marker], marker='|', color = 'cyan', linestyle='None')
#                         ax1.text(wavelength[wavelength_min_nii2-3], marker_threshold, 'NII', fontsize=9)
#                         ax1.plot(wavelength[wavelength_min_halpha],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
#                         ax1.text(wavelength[wavelength_min_halpha-6], marker_threshold, 'Halpha', fontsize=9)


#                         ax1.axvline(x = wavelength[x_0], linestyle = '--', color = 'r')
#                         ax1.axvline(x = wavelength[x_1], linestyle = '--', color = 'r')
#                         ax1.axvline(x = wavelength[x_1 + 5], linestyle = '-', color = 'k')
#                         ax1.axvline(x = wavelength[x_1_9 + 5], linestyle = '-', color = 'k')

#                         p_i = wavelength_bounds.shape[0]
#                         wavelength_range = np.arange(wavelength_bounds[0],wavelength_bounds[p_i-1], 0.25)

#                         ax1.plot(wavelength_bounds,double_gaussian(wavelength_bounds, *popt_dg), 'r-', label = 'Gaussian')

#                         ax1.legend(loc = 1)

#                         fig.savefig(newpath + iau + '_' + emline_name + '_dg.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0.1)
#                         #plt.show()
#                         plt.close()

#                         #Continuum Flux Maps
#                         f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
#                         cax1 = ax1.imshow(flux_galaxy, cmap='gist_heat', origin = 'lower', norm=LogNorm())
#                         ax1.set_title(iau + ' Continuum Galaxy Flux Map')
#                         ax1.set_xlabel('Spaxels')
#                         ax1.set_ylabel('Spaxels')
#                         f.colorbar(cax1, ax = ax1, label = 'Flux')
#                         vmin, vmax = cax1.get_clim()
#                         cax2 = ax2.imshow(flux_nuclear_sum, cmap='gist_heat', origin = 'lower', norm=LogNorm())
#                         ax2.set_title(iau + ' Continuum Nuclear Flux Map')
#                         ax2.set_xlabel('Spaxels')
#                         ax2.set_ylabel('Spaxels')
#                         f.colorbar(cax2, ax = ax2, label = 'Flux')
#                         f.savefig(newpath +  iau + '_' + 'continuum_flux_maps_dg.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
#                         #plt.show()
#                         plt.close()

#                         #Emission Line Flux Maps
#                         f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
#                         cax1 = ax1.imshow(flux_import[x_0:x_1, :, :].sum(axis = 0), cmap='gist_heat', origin = 'lower', norm=LogNorm(), vmax = vmax)
#                         ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
#                         ax1.set_xlabel('Spaxels')
#                         ax1.set_ylabel('Spaxels')
#                         f.colorbar(cax1, ax = ax1, label = 'Flux')
#                         cax2 = ax2.imshow(flux_nuclear_dimensions[x_0:x_1, :, :].sum(axis = (0)), cmap='gist_heat', origin = 'lower', norm=LogNorm(), vmax = vmax)
#                         ax2.set_title(iau  + ' ' + emline_name + ' ' + ' Nuclear Flux Map')
#                         ax2.set_xlabel('Spaxels')
#                         ax2.set_ylabel('Spaxels')
#                         f.colorbar(cax2, ax = ax2, label = 'Flux')
#                         f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps_dg.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
#                         #plt.show()
#                         plt.close() 


            velocity_meas = c*((popt[1] - emline)/emline)
            velocity_sys = c*tbdata['z'][ind][0]
            velocity_off = velocity_sys - velocity_meas
            sigma = c*(abs(popt[2])/emline)
            fwhm = sigma*2.355


            if emline == FeVII_3586 or emline == FeVII_3760 or emline == FeVII_6086:
                ip = 99.10  
            if emline == FeX_6374:
                ip = 233.60
            if emline == NeV_3346 or emline == NeV_3426:
                ip = 97.12

            # sigma_threshold = ((2000/2.355)/c)*(wavelength[wavelength_min])

            if sigma > 0 and sigma < 450 and popt[0] > flux_sigma and mu-15 < popt[1] < mu+15 and popt[0] > 3*perr[0]:

                newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/5_sigma/%s/'%emline_name + '%s/'%iau
                if not os.path.exists(newpath):
                    os.makedirs(newpath)
                f = open(newpath + "data.csv","w+")
                f.write('SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + '5 Sigma Threshold (ergs/cm^2/A/spaxel)'
                         + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
                        + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
                        + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
                        + 'Gaussian Sigma (A)' + ',' 
                        + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM (km/s)' + ',' + 'Measured Velocity (km/s)' 
                        + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
                         + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)'+
                        '\n' + str(iau) + ',' + str(tbdata['z'][ind][0]) + ',' + str(distance) + ',' + str(flux_sigma) + ',' 
                        + str(popt[0])+ ',' + str(perr[0]) 
                        + ','  + str(popt[1]) + ',' + str(perr[1]) +  ','  + str(popt[2]) + ',' + str(perr[2]) + ',' 
                        + str(sigma) + ','  + str(fwhm) + ',' + str(velocity_meas)+ ',' + str(smbh_mass) + ',' + str(sersic_mass) 
                        + ','  + str(velocity_dispersion) + ',' + str(sfr) + ',' + str(chi_squared) + ',' + str(reduced_chi_squared) 
                        + ',' + str(ip))
                f.close()

                fig = plt.figure(figsize=(15,10))
                ax1 = plt.axes()  
                ax2 = plt.axes([0.18, 0.7, 0.15, 0.15])
                ax1.plot(wavelength_bounds_total, flux_correction_total, 'b', label = 'Data')

                ax2.plot(wavelength_bounds, residuals, 'k')
                ax2.set_title('Residuals')
                ax2.set_ylabel('Flux')
                ax2.set_xlabel(r'Wavelength ($\AA$)')

                if len(fitsName) == 57:
                    ax1.set_title(iau + ' ('+ fitsName[13:22] + ')')
                if len(fitsName) == 58:
                    ax1.set_title(iau + ' ('+ fitsName[13:23] + ')')
                if len(fitsName) == 59:
                    ax1.set_title(iau + ' ('+ fitsName[13:24] + ')')

                ax1.set_ylabel('Flux', fontsize = 15)
                ax1.set_xlabel(r'Wavelength ($\AA$)', fontsize = 15)
                r = wavelength_bounds_total.shape
                ax1.set_xlim(wavelength_bounds_total[1],wavelength_bounds_total[r[0] - 1])

                marker = np.argmin(flux_correction_total)
                marker_min = np.argmax(flux_correction_total)
                marker_threshold = -(0.035*flux_correction_total[marker_min])

                ax1.text(wavelength[x_1 + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)

                if emline == FeVII_3586:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)

                if emline == FeVII_3760:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oii1],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oii1-1], marker_threshold, 'OII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oii2],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    #ax1.text(wavelength[wavelength_min_oii2-3], marker_threshold, fontsize=9)
                    ax1.plot(wavelength[wavelength_min_hei],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_hei-3], marker_threshold, 'OII', fontsize=9)


                if emline == FeVII_6086:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_na],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavelength_min_na-2], marker_threshold, 'Na', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi0],flux_correction_total[marker], marker='|', color = 'red', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi0-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_hei1],flux_correction_total[marker], marker='|', color = 'blue', linestyle='None')
                    ax1.text(wavelength[wavelength_min_hei1-2], marker_threshold, 'HeI', fontsize=9)

                if emline == FeX_6374:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-3], marker_threshold, 'FeX', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi2],flux_correction_total[marker], marker='|', color = 'purple', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi2-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_ni],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavelength_min_ni-2], marker_threshold, 'NI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nii1],flux_correction_total[marker], marker='|', color = 'magenta', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nii1-3], marker_threshold, 'NII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nii2],flux_correction_total[marker], marker='|', color = 'cyan', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nii2-3], marker_threshold, 'NII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_halpha],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                    ax1.text(wavelength[wavelength_min_halpha-6], marker_threshold, 'Halpha', fontsize=9)

#                             if emline == OIII_4:
#                                 plt.plot(wavelength[wavelength_min_oiii3],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
#                                 plt.text(wavelength[wavelength_min_oiii3-4], marker_threshold, 'OIII', fontsize=9)

                if emline == NeV_3346:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-3], marker_threshold, 'NeV', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nevi],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nevi-4], marker_threshold, 'Ne_VI', fontsize=9)

                if emline == NeV_3426:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-3], marker_threshold, 'NeV', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nev],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nev-3], marker_threshold, 'NeV', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oiii0],flux_correction_total[marker], marker='|', color = 'red', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oiii0-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_ni0],flux_correction_total[marker], marker='|', color = 'blue', linestyle='None')
                    ax1.text(wavelength[wavelength_min_ni0-2], marker_threshold, 'NI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_he0],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                    ax1.text(wavelength[wavelength_min_he0-2], marker_threshold, 'He', fontsize=9)


                ax1.axvline(x = wavelength[x_0], linestyle = '--', color = 'r')
                ax1.axvline(x = wavelength[x_1], linestyle = '--', color = 'r')
                 if emline == NeV_3426:
                    ax1.axvline(x = wavelength[x_1_9 + 5], linestyle = '-', color = 'k')
                    ax1.axvline(x = wavelength[x_1 + 8*bounds + 10], linestyle = '-', color = 'k')

                else:
                    ax1.axvline(x = wavelength[x_1 + 5], linestyle = '-', color = 'k')
                    ax1.axvline(x = wavelength[x_1_9 + 5], linestyle = '-', color = 'k')

                p_i = wavelength_bounds.shape[0]
                wavelength_range = np.arange(wavelength_bounds[0],wavelength_bounds[p_i-1], 0.25)

                ax1.plot(wavelength_range,gaus(wavelength_range, *popt), 'r-', label = 'Gaussian')
                ax1.legend(loc = 1)

                try:
                    fig.savefig(newpath + iau + '_' + emline_name + '.png', format='png', bbox_inches='tight', dpi=400, pad_inches = 0.1)
                except ValueError:
                    fig.savefig(newpath + iau + '_' + emline_name + '.png', format='png', dpi=400, pad_inches = 0.1)
                plt.close()

                #Continuum Flux Maps
                f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
                cax1 = ax1.imshow(flux_galaxy, cmap='gist_heat', origin = 'lower', norm=LogNorm())
                ax1.set_title(iau + ' Continuum Galaxy Flux Map')
                ax1.set_xlabel('Spaxels')
                ax1.set_ylabel('Spaxels')
                f.colorbar(cax1, ax = ax1, label = 'Flux')
                vmin, vmax = cax1.get_clim()
                cax2 = ax2.imshow(flux_nuclear_sum, cmap='gist_heat', origin = 'lower', norm=LogNorm())
                ax2.set_title(iau + ' Continuum Nuclear Flux Map')
                ax2.set_xlabel('Spaxels')
                ax2.set_ylabel('Spaxels')
                f.colorbar(cax2, ax = ax2, label = 'Flux')
                f.savefig(newpath +  iau + '_' + 'continuum_flux_maps.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
                #plt.show()
                plt.close()

                #Emission Line Flux Maps
                f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
                cax1 = ax1.imshow(flux_import[x_0:x_1, :, :].sum(axis = 0), cmap='gist_heat', origin = 'lower', norm=LogNorm(), vmax = vmax)
                ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
                ax1.set_xlabel('Spaxels')
                ax1.set_ylabel('Spaxels')
                f.colorbar(cax1, ax = ax1, label = 'Flux')
                cax2 = ax2.imshow(flux_nuclear_dimensions[x_0:x_1, :, :].sum(axis = (0)), cmap='gist_heat', origin = 'lower', norm=LogNorm(), vmax = vmax)
                ax2.set_title(iau  + ' ' + emline_name + ' ' + ' Nuclear Flux Map')
                ax2.set_xlabel('Spaxels')
                ax2.set_ylabel('Spaxels')
                f.colorbar(cax2, ax = ax2, label = 'Flux')
                f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
                #plt.show()
                plt.close() 



                     # get a map
                    #maps = Maps(plateifu=plateifu1)

                    # make a standard 3-plot BPT and retrieve the classifications
                    #masks, fig, axes = maps.get_bpt()

                    # save the plot
                    #fig.savefig(dir + iau + '_' + '3_plot_bpt.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0)

#                             # make a BPT classification without OI
#                             masks, fig, axes = maps.get_bpt(use_oi=False)
#                             # save the plot
#                             fig.savefig(dir + iau + 'no_OI_bpt.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0)

        hdulist.close()
        drpall.close()               

    return;


#         # Calling the function
#         emission(emline = FeVII_3760, 
#                  bounds = 15, 
#                  emline_name = 'FeVII_3760A')
#         emission(emline = FeVII_6086, 
#                  bounds = 15, 
#                  emline_name = 'FeVII_6086A')
#         emission(emline = NeV_3346, 
#                  bounds = 15,  
#                  emline_name = 'NeV_3346.79A')
#         emission(emline = FeVII_3586, 
#                  bounds = 15, 
#                  emline_name = 'FeVII_3586A')
#         emission(emline = NeV_3426, 
#                  bounds = 15,  
#                  emline_name = 'NeV_3426.85A')
#         emission(emline = FeX_6374, 
#                  bounds = 15, 
#                  emline_name = 'FeX_6374A')
#         emission(emline = H_Beta, 
#                 bounds = 15, 
#                 emline_name = 'HBeta_4862.68A')
#         emission(emline = OIII_4, 
#                 bounds = 15, 
#                 emline_name = 'OIII_4_5008.240A')

for fitsName in glob.glob('output/*.fits'):
    counter += 1
    print ('\n' + 'Galaxy Counter: ' + str(counter)) 
    os.remove(fitsName)                    



Galaxy Counter: 1


In [24]:

def emission(emline, bounds, emline_name):
    for fitsName in glob.glob('output/fits/%s/' %emline_name + '*.fits'):   
        hdulist = fits.open(fitsName) 
        wavelength_import = hdulist[4].data #Importing Wavelength Values
        flux_import = hdulist[1].data

        drpall = fits.open('drpall-v2_5_3.fits') #Opening drpall file
        tbdata = drpall[1].data #Accessing drpall data
        ind = np.where(tbdata['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
        iau = tbdata['nsa_iauname'][ind][0] #Finding SDSS Name
        print (iau)
        plateifu1 = tbdata['plateifu'][ind][0]
        sersic_mass = tbdata['nsa_sersic_mass'][ind][0] #Stellar mass from K-correction fit in h-2 solar masses to sersic magnitudes. 
        
        dapall = fits.open('dapall-v2_5_3-2.3.0.fits') #Opening dapall file
        tbdata_1 = dapall[1].data #Accessing dapall data
        ind_1 = np.where(tbdata_1['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
        velocity_dispersion = tbdata_1['STELLAR_SIGMA_1RE'][ind][0] 
        smbh_mass = (3.1*(velocity_dispersion/200)**4)*(10**8)
        distance = ((tbdata['z'][ind][0])*c)/H_0
        sfr = tbdata_1['SFR_1RE'][ind][0] 

        wavelength = wavelength_import/(tbdata['z'][ind][0] + 1)

        if emline == FeVII_3586 and tbdata['z'][ind][0] < 0.0136642:
            hdulist.close()
            drpall.close()  
            continue

        if emline == NeV_3346 and tbdata['z'][ind][0] < 0.0861153:
            hdulist.close()
            drpall.close()  
            continue

        index1,index2,index3 = flux_import.shape
        cp = int(index2/2)
        cp_down = cp-3
        cp_up = cp+3
        pd = int(index3/2)
        p_down = pd-3
        p_up = pd+3

        #Nuclear region flux
        flux_nuclear_dimensions = flux_import[:,cp_down:cp_up,p_down:p_up]
        flux_nuclear_sum = flux_nuclear_dimensions.sum(axis = (0))
        #flux = flux_nuclear_dimensions.sum(axis = (1, 2))

        #Galaxy flux
        flux_galaxy = flux_import.sum(axis = (0))
        flux = flux_import.sum(axis = (1,2))
        #flux_galaxy_1d = flux_import.sum(axis = (1,2))

        #Defining neighboring wavelengths
        wavelength_absolute = np.abs(wavelength - emline) 
        wavelength_min = np.argmin(wavelength_absolute)

        #FeVII_3760 Neighboring Lines
        wavelength_oii1 = np.abs(wavelength - OII_1) 
        wavelength_min_oii1 = np.argmin(wavelength_oii1)
        wavelength_oii2 = np.abs(wavelength - OII_2) 
        wavelength_min_oii2 = np.argmin(wavelength_oii2)
        wavelength_hei = np.abs(wavelength - He_I) 
        wavelength_min_hei = np.argmin(wavelength_hei)

        #FeVII_6086 Neighboring Lines
        wavelength_na = np.abs(wavelength - Na) 
        wavelength_min_na = np.argmin(wavelength_na) 
        wavelength_oi0 = np.abs(wavelength - OI_0) 
        wavelength_min_oi0 = np.argmin(wavelength_oi0) 
        wavelength_hei1 = np.abs(wavelength - He_I_1) 
        wavelength_min_hei1 = np.argmin(wavelength_hei1) 

        #FeX_6374 Neighboring Lines
        wavelength_oi1 = np.abs(wavelength - OI_1) 
        wavelength_min_oi1 = np.argmin(wavelength_oi1)
        wavelength_oi2 = np.abs(wavelength - OI_2) 
        wavelength_min_oi2 = np.argmin(wavelength_oi2)
        wavelength_ni = np.abs(wavelength - NI) 
        wavelength_min_ni = np.argmin(wavelength_ni)
        wavelength_nii1 = np.abs(wavelength - NII_1) 
        wavelength_min_nii1 = np.argmin(wavelength_nii1)
        wavelength_nii2 = np.abs(wavelength - NII_2) 
        wavelength_min_nii2 = np.argmin(wavelength_nii2)
        wavelength_halpha = np.abs(wavelength - H_Alpha) 
        wavelength_min_halpha = np.argmin(wavelength_halpha)

        #NeV_3346 Neighboring Lines
        wavelength_nevi = np.abs(wavelength - Ne_VI) 
        wavelength_min_nevi = np.argmin(wavelength_nevi)

        #H-Alpha Neighboring Lines
        wavelength_oiii3 = np.abs(wavelength - OIII_3) 
        wavelength_min_oiii3 = np.argmin(wavelength_oiii3)

        #Determining observed (for vmeasured value)
    #         lambda_obs = (tbdata['z'][ind][0] + 1)*emline
    #         wavelength_absolute = np.abs(wavelength - lambda_obs) 
    #         wavelength_min = np.argmin(wavelength_absolute)

        #Spectrum parameters
        x_0, x_1 = wavelength_min - bounds, wavelength_min + bounds #Setting lower bound for linear fit
        if x_0 < 0:
            x_0 = 0
        #Setting upper bound for linear fit
        z_1_2 = x_0 - 10*bounds
        if z_1_2 < 0:
            z_1_2 = 0
        x_1_2 = x_1 + 10*bounds

        wavelength_bounds_total = wavelength[z_1_2:x_1_2] 
        wavelength_total = np.abs(wavelength_bounds_total - emline) 
        wavelength_min_total = np.argmin(wavelength_total)

        wavelength_mask_total = np.ma.masked_array(wavelength_bounds_total,
                                             (wavelength_bounds_total != wavelength_bounds_total[0])& 
                                             (wavelength_bounds_total != wavelength_bounds_total[-1])) 
        wavelength_compressed_total = wavelength_mask_total.compressed() 

        flux_bounds_total = flux[z_1_2:x_1_2] 
        flux_mask_total = np.ma.masked_array(flux_bounds_total, (flux_bounds_total != flux_bounds_total[0]) 
                                   & (flux_bounds_total != flux_bounds_total[-1]))
        flux_compressed_total = flux_mask_total.compressed() 

        if flux_compressed_total.shape[0] == 3:
            print ('duplicate')
            flux_1,flux_2 = flux_compressed_total[0],flux_compressed_total[2]
            flux_compressed_total = [flux_1, flux_2]

        if flux_compressed_total[0] == 0:
            hdulist.close()
            drpall.close()  
            continue
                    
        else:
            polyfit_total = np.polyfit(wavelength_compressed_total,flux_compressed_total, 1) #Fitting the line to the data
            fit_total = np.poly1d(polyfit_total)
            flux_linear_total = fit_total[1]*wavelength_bounds_total + fit_total[0] #Determining linear fit parameters
            flux_correction_total = flux_bounds_total - flux_linear_total #Correcting for continuum

            for i in flux_correction_total:
                if i < 0:
                    negative = np.argmin(flux_correction_total)
                    flux_correction_total = flux_correction_total + abs(flux_correction_total[negative])
                    break

            q_0, q_1 = wavelength_min_total - bounds, wavelength_min_total + bounds
            if q_0 < 0:
                q_0 = 0
            wavelength_bounds = wavelength_bounds_total[q_0:q_1]
            flux_bounds = flux_correction_total[q_0:q_1]

            wavelength_bounds_min = np.abs(wavelength_bounds - emline) 
            wavelength_bounds_min_total = np.argmin(wavelength_bounds_min)

            wavelength_mask = np.ma.masked_array(wavelength_bounds,
                                             (wavelength_bounds != wavelength_bounds[0])& 
                                             (wavelength_bounds != wavelength_bounds[-1])) 
            wavelength_compressed = wavelength_mask.compressed() 

            flux_mask = np.ma.masked_array(flux_bounds, (flux_bounds != flux_bounds[0]) 
                                   & (flux_bounds != flux_bounds[-1]))
            flux_compressed = flux_mask.compressed() 

            if flux_compressed.shape[0] == 3:
                print ('duplicate')
                flux_compressed = [flux_1, flux_2]

            if flux_compressed[0] == 0:
                hdulist.close()
                drpall.close()  
                continue
            else:
                polyfit = np.polyfit(wavelength_compressed,flux_compressed, 1) #Fitting the line to the data
                fit = np.poly1d(polyfit)
                flux_linear = fit[1]*wavelength_bounds + fit[0] #Determining linear fit parameters
                avg_flux = np.average(flux_linear)

            x_1_9 = x_1 + 4*bounds
            flux_sigma_bounds_1 = flux[x_1 + 5: x_1_9 + 5]
            f_std = np.std(flux_sigma_bounds_1)
            flux_sigma = 5*(f_std)
            flux_sigma_1 = 1*(f_std)

            if len(fitsName) == 72:
                print ('\n' + iau + ' ('+ fitsName[28:37] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))
            if len(fitsName) == 73:
                print ('\n' + iau + ' ('+ fitsName[28:38] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))
            if len(fitsName) == 74:
                print ('\n' + iau + ' ('+ fitsName[30:39] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))
            if len(fitsName) == 75:
                print ('\n' + iau + ' (' + fitsName[30:40] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))
            if len(fitsName) == 76:
                print ('\n' + iau + ' (' + fitsName[30:41] + ') ' + emline_name + ' ' +
                '\n' + 'Five Sigma (ergs/cm^2/A/spaxel) = ' + str(flux_sigma))

            def gaus(x,amp,mu,sigma, m, c):
                return amp*np.exp(-(x-mu)**2/(2*sigma**2)) + m*x + c #Defining Gaussian function

            def double_gaussian(x,amp_ems,mu1,sigma1,m1,c1,amp2,mu2,sigma2,m2,c2):
                return gaus(x,amp_ems,mu1,sigma1,m1,c1) + gaus(x,amp2,mu2,sigma2,m2,c2)

            wavelength_emission = np.argmax(flux_bounds)
            amp_ems = flux_bounds[wavelength_emission]
            wavelength_absorption = np.argmin(flux_bounds)
            amp_abs = flux_bounds[wavelength_absorption]
            amp_abs_1 = -(avg_flux - amp_abs)
            sigma_abs_1 = avg_flux - flux_sigma_1
            sigma_abs_5 = avg_flux - flux_sigma
            sigma_ems = avg_flux + flux_sigma

            o1_wavelength_total = np.abs(wavelength_bounds - OI_2) 
            o1_wavelength_min_total = np.argmin(o1_wavelength_total)
            amp_o1 = flux_bounds[o1_wavelength_min_total]
            mu_o1 = wavelength_bounds[o1_wavelength_min_total]
            sigma_guess_o1 = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_o1)**2)/np.sum(flux_bounds))

            mu_abs = wavelength_bounds[wavelength_absorption]
            mu_ems = wavelength_bounds[wavelength_emission]
            mu = np.sum(wavelength_bounds*flux_bounds)/np.sum(flux_bounds)
            
            sigma_guess_abs = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_abs)**2)/np.sum(flux_bounds))
            sigma_guess_ems = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_ems)**2)/np.sum(flux_bounds))
            sigma_guess = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu)**2)/np.sum(flux_bounds))
            amp = flux_bounds[wavelength_bounds_min_total]

            if amp_abs < sigma_abs_5:
                hdulist.close()
                drpall.close() 
                print ('5 Sigma Absorption Line Error')
                continue

            if amp_abs < sigma_abs_1 and amp_ems < sigma_ems:

                try:
                    popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                               p0=[amp_abs_1, mu, sigma_guess, fit[1], 
                                   fit[0]])
                except RuntimeError:
                    popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                               p0=[amp_abs_1, mu, sigma_guess, 1, 
                                   1])

                print (emline_name + ' Absorption Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))

                if popt[0] < 0:
                    hdulist.close()
                    drpall.close() 
                    print ('Absorption Line Error')
                    continue

                else:
                    try:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                   p0=[amp_ems, mu, sigma_guess, fit[1], 
                                       fit[0]])
                    except RuntimeError:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                   p0=[amp_ems, mu, sigma_guess, 1, 
                                       1])

                    try:
                        perr = np.sqrt(np.diag(pcov))
                    except RuntimeWarning:
                        print ('RuntimeWarning')
                        hdulist.close()
                        drpall.close()
                        continue
                        
                    residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata

                    chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                    reduced_chi_squared = chi_squared / (len(popt) - 1)
                        
                    print (emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))
                    
#                     print ('skewness scipy = ' + str(skew(gaus(wavelength_bounds, *popt))))

                    I = np.sum(gaus(wavelength_bounds, *popt))
                    x_bar = (1/I)*np.sum(wavelength_bounds*gaus(wavelength_bounds, *popt))
                    std_1 = np.sqrt((1/I)*np.sum((wavelength_bounds - x_bar)**2*gaus(wavelength_bounds, *popt)))
                    S = (1/(I*std_1**3))*np.sum((wavelength_bounds - x_bar)**3*gaus(wavelength_bounds, *popt))

            else:
                try:
                    popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                               p0=[amp_ems, mu, sigma_guess, fit[1], 
                                   fit[0]])
                except RuntimeError:
                    hdulist.close()
                    drpall.close()
                    print ('RuntimeError ')
                    continue
                    

                try:
                    perr = np.sqrt(np.diag(pcov))
                except RuntimeWarning:
                    perr = 0
                    print ('RuntimeWarning')
                    hdulist.close()
                    drpall.close()
                    continue
                    
                residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata

                chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                reduced_chi_squared = chi_squared / (len(popt) - 1)

#                 print ('Chi Squared test = ' +  str(chi_squared) + '\n' + 'Reduced Chi Squared = ' 
#                        + str(reduced_chi_squared))
                        
                print (emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))
                
#                 print ('skewness scipy = ' + str(skew(gaus(wavelength_bounds, *popt))))

                I = np.sum(gaus(wavelength_bounds, *popt))
                x_bar = (1/I)*np.sum(wavelength_bounds*gaus(wavelength_bounds, *popt))
                std_1 = np.sqrt((1/I)*np.sum((wavelength_bounds - x_bar)**2*gaus(wavelength_bounds, *popt)))
                S = (1/(I*std_1**3))*np.sum((wavelength_bounds - x_bar)**3*gaus(wavelength_bounds, *popt))
                #print ('skewness lit = ' + str(S))

#                 if emline == FeX_6374:    
#                     try:
#                         popt_dg, pcov_dg = curve_fit(double_gaussian,wavelength_bounds, flux_bounds,  
#                                        p0=[amp,mu,sigma_guess,fit[1],fit[0],amp_o1,mu_o1,sigma_guess_o1,fit[1],fit[0]]) 
#                     except RuntimeError:
#                         hdulist.close()
#                         drpall.close() 
#                         print ('RuntimeError')
#                         continue

#                     try:
#                         perr_dg = np.sqrt(np.diag(pcov_dg))
#                     except RuntimeWarning:
#                         hdulist.close()
#                         drpall.close()
#                         print ('RuntimeWarning')
#                         continue
                        
#                     residuals_dg = flux_bounds - double_gaussian(wavelength_bounds, *popt_dg) #Determines the uncertainty in ydata

#                     chi_squared_dg =  np.sum(((residuals_dg)** 2)/(double_gaussian(wavelength_bounds, *popt_dg)))
#                     reduced_chi_squared_dg = chi_squared_dg / (len(popt_dg) - 1)

#                     print ('Double Gaussian Chi Squared = ' +  str(chi_squared_dg) + '\n' + 'Double Gaussian Reduced Chi Squared = ' 
#                            + str(reduced_chi_squared_dg))

#                     print (emline_name + ' OI Flux (ergs/cm^2/A/spaxel) = ' + str(popt_dg[5]) + '\n'
#                            + emline_name + ' FeX Flux (ergs/cm^2/A/spaxel) = ' + str(popt_dg[0]))

#                     velocity_meas_dg = c*((popt_dg[1] - emline)/emline)
#                     velocity_sys_dg = c*tbdata['z'][ind][0]
#                     velocity_off_dg = velocity_sys_dg - velocity_meas_dg
#                     sigma_dg = c*(abs(popt_dg[2])/emline)
#                     fwhm_dg = sigma_dg*2.355
#                     ip = 262.1
                    
#                     #if sigma_dg > 0 and sigma_dg < 450 and popt_dg[0] > flux_sigma and mu-15 < popt_dg[1] < mu+15 and popt_dg[0] > 3*perr_dg[0]:

#                     newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/5_sigma/%s/'%emline_name + '%s/'%iau
#                     if not os.path.exists(newpath):
#                         os.makedirs(newpath)
#                     f = open(newpath + "dg_data.csv","w+")
#                     f.write('SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + '5 Sigma Threshold (ergs/cm^2/A/spaxel)'
#                              + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
#                             + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
#                             + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
#                             + 'Gaussian Sigma (A)' + ',' 
#                             + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM (km/s)' + ',' + 'Measured Velocity (km/s)' 
#                             + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
#                              + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)'+
#                             '\n' + str(iau) + ',' + str(tbdata['z'][ind][0]) + ',' + str(distance) + ',' + str(flux_sigma) + ',' 
#                             + str(popt_dg[0])+ ',' + str(perr_dg[0]) 
#                             + ','  + str(popt_dg[1]) + ',' + str(perr_dg[1]) +  ','  + str(popt_dg[2]) + ',' + str(perr_dg[2]) + ',' 
#                             + str(sigma_dg) + ','  + str(fwhm_dg) + ',' + str(velocity_meas_dg)+ ',' + str(smbh_mass) + ',' + str(sersic_mass) 
#                             + ','  + str(velocity_dispersion) + ',' + str(sfr) + ',' + str(chi_squared_dg) + ',' + str(reduced_chi_squared_dg) 
#                             + ',' + str(ip))
#                     f.close()

#                     fig = plt.figure(figsize=(15,10))
#                     ax1 = plt.axes()  
#                     ax2 = plt.axes([0.18, 0.7, 0.15, 0.15])
#                     ax1.plot(wavelength_bounds_total, flux_correction_total, 'b', label = 'Data')

#                     ax2.plot(wavelength_bounds, residuals, 'k')
#                     ax2.set_title('Residuals')
#                     ax2.set_ylabel('Flux')
#                     ax2.set_xlabel(r'Wavelength ($\AA$)')

#                     if len(fitsName) == 57:
#                         ax1.set_title(iau + ' ('+ fitsName[13:22] + ')')
#                     if len(fitsName) == 58:
#                         ax1.set_title(iau + ' ('+ fitsName[13:23] + ')')
#                     if len(fitsName) == 59:
#                         ax1.set_title(iau + ' ('+ fitsName[13:24] + ')')

#                     ax1.set_ylabel('Flux', fontsize = 15)
#                     ax1.set_xlabel(r'Wavelength ($\AA$)', fontsize = 15)
#                     r = wavelength_bounds_total.shape
#                     ax1.set_xlim(wavelength_bounds_total[1],wavelength_bounds_total[r[0] - 1])

#                     marker = np.argmin(flux_correction_total)
#                     marker_min = np.argmax(flux_correction_total)
#                     marker_threshold = -(0.035*flux_correction_total[marker_min])

#                     ax1.text(wavelength[x_1 + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)


#                     ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
#                     ax1.text(wavelength[wavelength_min-3], marker_threshold, 'FeX', fontsize=9)
#                     ax1.plot(wavelength[wavelength_min_oi2],flux_correction_total[marker], marker='|', color = 'purple', linestyle='None')
#                     ax1.text(wavelength[wavelength_min_oi2-2], marker_threshold, 'OI', fontsize=9)
#                     ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
#                     ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
#                     ax1.plot(wavelength[wavelength_min_ni],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
#                     ax1.text(wavelength[wavelength_min_ni-2], marker_threshold, 'NI', fontsize=9)
#                     ax1.plot(wavelength[wavelength_min_nii1],flux_correction_total[marker], marker='|', color = 'magenta', linestyle='None')
#                     ax1.text(wavelength[wavelength_min_nii1-3], marker_threshold, 'NII', fontsize=9)
#                     ax1.plot(wavelength[wavelength_min_nii2],flux_correction_total[marker], marker='|', color = 'cyan', linestyle='None')
#                     ax1.text(wavelength[wavelength_min_nii2-3], marker_threshold, 'NII', fontsize=9)
#                     ax1.plot(wavelength[wavelength_min_halpha],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
#                     ax1.text(wavelength[wavelength_min_halpha-6], marker_threshold, 'Halpha', fontsize=9)


#                     ax1.axvline(x = wavelength[x_0], linestyle = '--', color = 'r')
#                     ax1.axvline(x = wavelength[x_1], linestyle = '--', color = 'r')
#                     ax1.axvline(x = wavelength[x_1 + 5], linestyle = '-', color = 'k')
#                     ax1.axvline(x = wavelength[x_1_9 + 5], linestyle = '-', color = 'k')

#                     p_i = wavelength_bounds.shape[0]
#                     wavelength_range = np.arange(wavelength_bounds[0],wavelength_bounds[p_i-1], 0.25)

#                     ax1.plot(wavelength_bounds,double_gaussian(wavelength_bounds, *popt_dg), 'r-', label = 'Gaussian')

#                     ax1.legend(loc = 1)

#                     fig.savefig(newpath + iau + '_' + emline_name + '_dg.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0.1)
#                     #plt.show()
#                     plt.close()

#                     #Continuum Flux Maps
#                     f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
#                     cax1 = ax1.imshow(flux_galaxy, cmap='gist_heat', origin = 'lower', norm=LogNorm())
#                     ax1.set_title(iau + ' Continuum Galaxy Flux Map')
#                     ax1.set_xlabel('Spaxels')
#                     ax1.set_ylabel('Spaxels')
#                     f.colorbar(cax1, ax = ax1, label = 'Flux')
#                     vmin, vmax = cax1.get_clim()
#                     cax2 = ax2.imshow(flux_nuclear_sum, cmap='gist_heat', origin = 'lower', norm=LogNorm())
#                     ax2.set_title(iau + ' Continuum Nuclear Flux Map')
#                     ax2.set_xlabel('Spaxels')
#                     ax2.set_ylabel('Spaxels')
#                     f.colorbar(cax2, ax = ax2, label = 'Flux')
#                     f.savefig(newpath +  iau + '_' + 'continuum_flux_maps_dg.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
#                     #plt.show()
#                     plt.close()

#                     #Emission Line Flux Maps
#                     f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
#                     cax1 = ax1.imshow(flux_import[x_0:x_1, :, :].sum(axis = 0), cmap='gist_heat', origin = 'lower', norm=LogNorm(), vmax = vmax)
#                     ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
#                     ax1.set_xlabel('Spaxels')
#                     ax1.set_ylabel('Spaxels')
#                     f.colorbar(cax1, ax = ax1, label = 'Flux')
#                     cax2 = ax2.imshow(flux_nuclear_dimensions[x_0:x_1, :, :].sum(axis = (0)), cmap='gist_heat', origin = 'lower', norm=LogNorm(), vmax = vmax)
#                     ax2.set_title(iau  + ' ' + emline_name + ' ' + ' Nuclear Flux Map')
#                     ax2.set_xlabel('Spaxels')
#                     ax2.set_ylabel('Spaxels')
#                     f.colorbar(cax2, ax = ax2, label = 'Flux')
#                     f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps_dg.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
#                     #plt.show()
#                     plt.close() 

                        
            velocity_meas = c*((popt[1] - emline)/emline)
            velocity_sys = c*tbdata['z'][ind][0]
            velocity_off = velocity_sys - velocity_meas
            sigma = c*(abs(popt[2])/emline)
            fwhm = sigma*2.355
            
            if emline == FeVII_3586 or FeVII_3760 or FeVII_6086:
                ip = 99.10  
            if emline == FeX_6374:
                ip = 233.60
            if emline == NeV_3346:
                ip = 97.12
                
            # sigma_threshold = ((2000/2.355)/c)*(wavelength[wavelength_min])
            if sigma > 0 and sigma < 450 and popt[0] > flux_sigma and mu-15 < popt[1] < mu+15 and popt[0] > 3*perr[0]:

                newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/5_sigma/%s/'%emline_name + '%s/'%iau
                if not os.path.exists(newpath):
                    os.makedirs(newpath)
                f = open(newpath + "data.csv","w+")
                f.write('SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + '5 Sigma Threshold (ergs/cm^2/A/spaxel)'
                                 + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
                                + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
                                + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
                                + 'Gaussian Sigma (A)' + ',' 
                                + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM' + ',' + 'Measured Velocity (km/s)' 
                                + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
                                 + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)'+ ',' 
                                + 'Skewness' +
                                '\n' + str(iau) + ',' + str(tbdata['z'][ind][0]) + ',' + str(distance) + ',' + str(flux_sigma) + ',' 
                                + str(popt[0])+ ',' + str(perr[0]) 
                                + ','  + str(popt[1]) + ',' + str(perr[1]) +  ','  + str(popt[2]) + ',' + str(perr[2]) + ',' 
                                + str(sigma) + ','  + str(fwhm) + ',' + str(velocity_meas)+ ',' + str(smbh_mass) + ',' + str(sersic_mass) 
                                + ','  + str(velocity_dispersion) + ',' + str(sfr) + ',' + str(chi_squared) + ',' + str(reduced_chi_squared) 
                                + ',' + str(ip) + ',' + str(S))
                f.close()

                fig = plt.figure(figsize=(15,10))
                ax1 = plt.axes()  
                ax2 = plt.axes([0.18, 0.7, 0.15, 0.15])
                ax1.plot(wavelength_bounds_total, flux_correction_total, 'b', label = 'Data')

                ax2.plot(wavelength_bounds, residuals, 'k')
                ax2.set_title('Residuals')
                ax2.set_ylabel('Flux')
                ax2.set_xlabel(r'Wavelength ($\AA$)')

                if len(fitsName) == 72:
                    ax1.set_title(iau + ' ('+ fitsName[28:37] + ')')
                if len(fitsName) == 73:
                    ax1.set_title(iau + ' ('+ fitsName[28:38] + ')')
                if len(fitsName) == 74:
                    ax1.set_title(iau + ' ('+ fitsName[30:39] + ')')
                if len(fitsName) == 75:
                    ax1.set_title(iau + ' ('+ fitsName[30:40] + ')')
                if len(fitsName) == 76:
                    ax1.set_title(iau + ' ('+ fitsName[30:41] + ')')

                ax1.set_ylabel('Flux', fontsize = 15)
                ax1.set_xlabel(r'Wavelength ($\AA$)', fontsize = 15)
                r = wavelength_bounds_total.shape
                ax1.set_xlim(wavelength_bounds_total[1],wavelength_bounds_total[r[0] - 1])

                marker = np.argmin(flux_correction_total)
                marker_min = np.argmax(flux_correction_total)
                marker_threshold = -(0.035*flux_correction_total[marker_min])

                ax1.text(wavelength[x_1 + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)

                if emline == FeVII_3586:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)

                if emline == FeVII_3760:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oii1],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oii1-1], marker_threshold, 'OII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oii2],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    #ax1.text(wavelength[wavelength_min_oii2-3], marker_threshold, fontsize=9)
                    ax1.plot(wavelength[wavelength_min_hei],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_hei-3], marker_threshold, 'He', fontsize=9)


                if emline == FeVII_6086:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_na],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavelength_min_na-2], marker_threshold, 'Na', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi0],flux_correction_total[marker], marker='|', color = 'red', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi0-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_hei1],flux_correction_total[marker], marker='|', color = 'blue', linestyle='None')
                    ax1.text(wavelength[wavelength_min_hei1-2], marker_threshold, 'HeI', fontsize=9)

                    

                if emline == FeX_6374:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-3], marker_threshold, 'FeX', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi2],flux_correction_total[marker], marker='|', color = 'purple', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi2-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_ni],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavelength_min_ni-2], marker_threshold, 'NI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nii1],flux_correction_total[marker], marker='|', color = 'magenta', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nii1-3], marker_threshold, 'NII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nii2],flux_correction_total[marker], marker='|', color = 'cyan', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nii2-3], marker_threshold, 'NII', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_halpha],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                    ax1.text(wavelength[wavelength_min_halpha-6], marker_threshold, 'Halpha', fontsize=9)

    #                             if emline == OIII_4:
    #                                 plt.plot(wavelength[wavelength_min_oiii3],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
    #                                 plt.text(wavelength[wavelength_min_oiii3-4], marker_threshold, 'OIII', fontsize=9)

                if emline == NeV_3346:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-3], marker_threshold, 'NeV', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nevi],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nevi-4], marker_threshold, 'Ne_VI', fontsize=9)
                    
                if emline == NeV_3426:
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-3], marker_threshold, 'NeV', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_nev],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavelength_min_nev-3], marker_threshold, 'NeV', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_oiii0],flux_correction_total[marker], marker='|', color = 'red', linestyle='None')
                    ax1.text(wavelength[wavelength_min_oiii0-2], marker_threshold, 'OI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_ni0],flux_correction_total[marker], marker='|', color = 'blue', linestyle='None')
                    ax1.text(wavelength[wavelength_min_ni0-2], marker_threshold, 'NI', fontsize=9)
                    ax1.plot(wavelength[wavelength_min_he0],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                    ax1.text(wavelength[wavelength_min_he0-2], marker_threshold, 'He', fontsize=9)

                ax1.axvline(x = wavelength[x_0], linestyle = '--', color = 'r')
                ax1.axvline(x = wavelength[x_1], linestyle = '--', color = 'r')
                ax1.axvline(x = wavelength[x_1 + 5], linestyle = '-', color = 'k')
                ax1.axvline(x = wavelength[x_1_9 + 5], linestyle = '-', color = 'k')

                p_i = wavelength_bounds.shape[0]
                wavelength_range = np.arange(wavelength_bounds[0],wavelength_bounds[p_i-1], 0.25)

                if len(popt) == 10:
                    ax1.plot(wavelength_range,double_gaussian(wavelength_range, *popt), 'r-', label = 'Gaussian')
                else:
                    ax1.plot(wavelength_range,gaus(wavelength_range, *popt), 'r-', label = 'Gaussian')
                ax1.legend(loc = 1)

                fig.savefig(newpath + iau + '_' + emline_name + '.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0.1)
                #plt.show()
                plt.close()

                #Emission Line Flux Maps
                f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
                cax1 = ax1.imshow(flux_import[x_0:x_1, :, :].sum(axis = 0), cmap='gist_heat', origin = 'lower')
                ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
                ax1.set_xlabel('Spaxels')
                ax1.set_ylabel('Spaxels')
                f.colorbar(cax1, ax = ax1, label = 'Flux')
                cax2 = ax2.imshow(flux_nuclear_dimensions[x_0:x_1, :, :].sum(axis = (0)), cmap='gist_heat', origin = 'lower')
                ax2.set_title(iau  + ' ' + emline_name + ' ' + ' Nuclear Flux Map')
                ax2.set_xlabel('Spaxels')
                ax2.set_ylabel('Spaxels')
                f.colorbar(cax2, ax = ax2, label = 'Flux')
                f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
                #plt.show()
                plt.close()

                #Continuum Flux Maps
                f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(20,8))
                cax1 = ax1.imshow(flux_galaxy, cmap='gist_heat', origin = 'lower')
                ax1.set_title(iau + ' Continuum Galaxy Flux Map')
                ax1.set_xlabel('Spaxels')
                ax1.set_ylabel('Spaxels')
                f.colorbar(cax1, ax = ax1, label = 'Flux')
                cax2 = ax2.imshow(flux_nuclear_sum, cmap='gist_heat', origin = 'lower')
                ax2.set_title(iau + ' Continuum Nuclear Flux Map')
                ax2.set_xlabel('Spaxels')
                ax2.set_ylabel('Spaxels')
                f.colorbar(cax2, ax = ax2, label = 'Flux')
                f.savefig(newpath +  iau + '_' + 'continuum_flux_maps.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
                plt.close()


                 # get a map
                #maps = Maps(plateifu=plateifu1)

                # make a standard 3-plot BPT and retrieve the classifications
                #masks, fig, axes = maps.get_bpt()

                # save the plot
                #fig.savefig(dir + iau + '_' + '3_plot_bpt.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0)

    #                             # make a BPT classification without OI
    #                             masks, fig, axes = maps.get_bpt(use_oi=False)
    #                             # save the plot
    #                             fig.savefig(dir + iau + 'no_OI_bpt.png', dpi=400, format='png', bbox_inches='tight', pad_inches = 0)

            hdulist.close()
            drpall.close()               

    return;


# Calling the function 
# emission(emline = FeVII_3760, 
#          bounds = 15, 
#          emline_name = 'FeVII_3760A')
# emission(emline = FeVII_6086, 
#          bounds = 15, 
#          emline_name = 'FeVII_6086A')
# emission(emline = NeV_3346, 
#          bounds = 15,  
#          emline_name = 'NeV_3346.79A')
# emission(emline = FeVII_3586, 
#          bounds = 15, 
#          emline_name = 'FeVII_3586A')
# emission(emline = FeX_6374, 
#          bounds = 15, 
#          emline_name = 'FeX_6374A')
#         emission(emline = H_Beta, 
#                 bounds = 15, 
#                 emline_name = 'HBeta_4862.68A')#         emission(emline = OIII_4, 
#                 bounds = 15, 
#                 emline_name = 'OIII_4_5008.240A')



In [152]:
def emission(emline, bounds, emline_name):
    
    for fitsName in glob.glob('output/fits/%s/' %emline_name + 'logcube/' + '*.fits'): 
        hdulist = fits.open(fitsName) 
        hdu = fits.open('zoo2MainSpecz.fits')
        drpall = fits.open('drpall-v2_5_3.fits') 
        dapall = fits.open('dapall-v2_5_3-2.3.0.fits') 
        
        tbdata = drpall[1].data 
        ind = np.where(tbdata['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
        iau = tbdata['nsa_iauname'][ind][0] #Finding SDSS Name
        plateifu1 = tbdata['plateifu'][ind][0]
        redshift = tbdata['z'][ind][0]
        
        tbdata_1 = dapall[1].data 
        ind_1 = np.where(tbdata_1['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
    
        galaxy_zoo = hdu[1].data
        ra = str(iau[1:3]) + ':' + str(iau[3:5] + ':' + str(iau[5:7]) + iau[7:10]) 
        dec = str(iau[10:13]) + ':' + str(iau[13:15] + ':' + str(iau[15:17]) + iau[17:19]) 
        ind_2 = np.where(galaxy_zoo['rastring'] == ra) #Finding MaNGA ID
        galaxy_type = galaxy_zoo['gz2class'][ind_2]
        
        wavelength_import = hdulist[4].data #Importing Wavelength Values
        wavelength = wavelength_import/(tbdata['z'][ind][0] + 1)
        flux_import = hdulist[1].data
        
        velocity_dispersion = tbdata_1['STELLAR_SIGMA_1RE'][ind][0] 
        smbh_mass = (3.1*(velocity_dispersion/200)**4)*(10**8)
        sersic_mass = tbdata['nsa_sersic_mass'][ind][0] #Stellar mass from K-correction fit in h-2 solar masses to sersic magnitudes.
        distance = ((tbdata['z'][ind][0])*c)/H_0
        spaxel_size = (2*math.tan(0.25)/206265)*(distance*10**6)
        sfr = tbdata_1['SFR_1RE'][ind][0] 
        
        print (str(iau) + ' ' + str(emline_name))
        
        if emline == FeVII_3586 and redshift < 0.0156:
            hdulist.close()
            drpall.close()  
            continue

        if emline == NeV_3346 and redshift < 0.0882:
            hdulist.close()
            drpall.close()  
            continue
            
        if emline == NeV_3426 and redshift < 0.0628:
            hdulist.close()
            drpall.close()  
            continue

        index1,index2,index3 = flux_import.shape
        cp = int(index2/2)
        cp_down = cp-3
        cp_up = cp+3
        pd = int(index3/2)
        p_down = pd-3
        p_up = pd+3

        #Galaxy flux
        flux_galaxy = flux_import.sum(axis = (0))
        #flux = flux_import.sum(axis = (1,2))
        #flux_galaxy_1d = flux_import.sum(axis = (1,2))
        
        x,y,z = flux_import.shape
        print ('Total Array Size = ' + str(y) + ' x ' + str(z) + '\n')
        shape = z/2 + 1
        
        for i in range (1, int(shape)):
            flux_nuclear_dimensions = flux_import[:,cp - i:cp + i,pd - i:pd + i]
            h,u,v = flux_nuclear_dimensions.shape
            print ('Current Array Size = ' + str(u) + ' x ' +  str(v))
            
            for k in range (0, h):
                for j in range (1,v-1):
                    flux_nuclear_dimensions[k][j][1:v-1] = 0
            print (flux_nuclear_dimensions)
            flux = flux_nuclear_dimensions.sum(axis = (1, 2))
            print (flux)
            
            #Defining neighboring wavelengths
            wavelength_absolute = np.abs(wavelength - emline) 
            wavelength_min = np.argmin(wavelength_absolute)

            #FeVII_3076 Neighboring Lines
            wavelength_oii1 = np.abs(wavelength - OII_1) 
            wavelength_min_oii1 = np.argmin(wavelength_oii1)
            wavelength_oii2 = np.abs(wavelength - OII_2) 
            wavelength_min_oii2 = np.argmin(wavelength_oii2)
            wavelength_hei = np.abs(wavelength - He_I) 
            wavelength_min_hei = np.argmin(wavelength_hei)

            #FeVII_6086 Neighboring Lines
            wavelength_na = np.abs(wavelength - Na) 
            wavelength_min_na = np.argmin(wavelength_na) 
            wavelength_oi0 = np.abs(wavelength - OI_0) 
            wavelength_min_oi0 = np.argmin(wavelength_oi0) 
            wavelength_hei1 = np.abs(wavelength - He_I_1) 
            wavelength_min_hei1 = np.argmin(wavelength_hei1) 

            #FeX_6374 Neighboring Lines
            wavelength_oi1 = np.abs(wavelength - OI_1) 
            wavelength_min_oi1 = np.argmin(wavelength_oi1)
            wavelength_oi2 = np.abs(wavelength - OI_2) 
            wavelength_min_oi2 = np.argmin(wavelength_oi2)
            wavelength_ni = np.abs(wavelength - NI) 
            wavelength_min_ni = np.argmin(wavelength_ni)
            wavelength_nii1 = np.abs(wavelength - NII_1) 
            wavelength_min_nii1 = np.argmin(wavelength_nii1)
            wavelength_nii2 = np.abs(wavelength - NII_2) 
            wavelength_min_nii2 = np.argmin(wavelength_nii2)
            wavelength_halpha = np.abs(wavelength - H_Alpha) 
            wavelength_min_halpha = np.argmin(wavelength_halpha)

            #NeV_3346 Neighboring Lines
            wavelength_nevi = np.abs(wavelength - NeV_3426) 
            wavelength_min_nevi = np.argmin(wavelength_nevi)

            #NeV_3426 Neighboring Lines
            wavelength_nev = np.abs(wavelength - NeV_3346) 
            wavelength_min_nev = np.argmin(wavelength_nev)
            wavelength_oiii0 = np.abs(wavelength - OIII_0) 
            wavelength_min_oiii0 = np.argmin(wavelength_oiii0)
            wavelength_ni0 = np.abs(wavelength - NI_0) 
            wavelength_min_ni0 = np.argmin(wavelength_ni0)
            wavelength_he0 = np.abs(wavelength - He_0) 
            wavelength_min_he0 = np.argmin(wavelength_he0)

            #H-Alpha Neighboring Lines
            wavelength_oiii3 = np.abs(wavelength - OIII_3) 
            wavelength_min_oiii3 = np.argmin(wavelength_oiii3)

            #Determining observed (for vmeasured value)
        #         lambda_obs = (tbdata['z'][ind][0] + 1)*emline
        #         wavelength_absolute = np.abs(wavelength - lambda_obs) 
        #         wavelength_min = np.argmin(wavelength_absolute)

            
            print 
            #Spectrum parameters
            x_0, x_1 = wavelength_min - bounds, wavelength_min + bounds #Setting lower bound for linear fit
            if x_0 < 0:
                x_0 = 0
            #Setting upper bound for linear fit
            z_1_2 = x_0 - 10*bounds
            if z_1_2 < 0:
                z_1_2 = 0
            x_1_2 = x_1 + 10*bounds

            wavelength_bounds_total = wavelength[z_1_2:x_1_2] 
            
            wavelength_total = np.abs(wavelength_bounds_total - emline) 
            try: 
                wavelength_min_total = np.argmin(wavelength_total)
            except ValueError:
                print ('Empty Array' + '\n')
                continue
                
            wavelength_mask_total = np.ma.masked_array(wavelength_bounds_total,
                                                 (wavelength_bounds_total != wavelength_bounds_total[0])& 
                                                 (wavelength_bounds_total != wavelength_bounds_total[-1])) 
            wavelength_compressed_total = wavelength_mask_total.compressed() 

            flux_bounds_total = flux[z_1_2:x_1_2] 
            flux_mask_total = np.ma.masked_array(flux_bounds_total, (flux_bounds_total != flux_bounds_total[0]) 
                                       & (flux_bounds_total != flux_bounds_total[-1]))
            flux_compressed_total = flux_mask_total.compressed() 

            if flux_compressed_total.shape[0] > 2:
                flux_1,flux_2 = flux_compressed_total[0],flux_compressed_total[-1]
                flux_compressed_total = [flux_1, flux_2]

            if flux_compressed_total[0] == 0:
                print ('No Spectrum Available' + '\n')
                continue
            
            else:
                
                polyfit_total = np.polyfit(wavelength_compressed_total,flux_compressed_total, 1) #Fitting the line to the data
                fit_total = np.poly1d(polyfit_total)
                flux_linear_total = fit_total[1]*wavelength_bounds_total + fit_total[0] #Determining linear fit parameters
                flux_correction_total = flux_bounds_total - flux_linear_total #Correcting for continuum


                for i in flux_correction_total:
                    if i < 0:
                        negative = np.argmin(flux_correction_total)
                        flux_correction_total = flux_correction_total + abs(flux_correction_total[negative])
                        break

                q_0, q_1 = wavelength_min_total - bounds, wavelength_min_total + bounds
                if q_0 < 0:
                    q_0 = 0
                wavelength_bounds = wavelength_bounds_total[q_0:q_1]
                flux_bounds = flux_correction_total[q_0:q_1]

                wavelength_bounds_min = np.abs(wavelength_bounds - emline) 
                wavelength_bounds_min_total = np.argmin(wavelength_bounds_min)

                wavelength_mask = np.ma.masked_array(wavelength_bounds,
                                                 (wavelength_bounds != wavelength_bounds[0])& 
                                                 (wavelength_bounds != wavelength_bounds[-1])) 
                wavelength_compressed = wavelength_mask.compressed() 

                flux_mask = np.ma.masked_array(flux_bounds, (flux_bounds != flux_bounds[0]) 
                                       & (flux_bounds != flux_bounds[-1]))
                flux_compressed = flux_mask.compressed() 

                if flux_compressed.shape[0] > 2:
                    flux_1,flux_2 = flux_compressed[0],flux_compressed[-1]
                    flux_compressed = [flux_1, flux_2]

                if flux_compressed[0] == 0:
                    print ('No Spectrum Available' + '\n')
                    continue
                    
                else:
                    polyfit = np.polyfit(wavelength_compressed,flux_compressed, 1) #Fitting the line to the data
                    fit = np.poly1d(polyfit)
                    flux_linear = fit[1]*wavelength_bounds + fit[0] #Determining linear fit parameters
                    avg_flux = np.average(flux_linear)

                x_1_9 = x_1 + 4*bounds
    
                if emline == FeX_6374 or emline == FeVII_3760 or emline == FeVII_6086:
                    x_test_1 = x_1 - 9*bounds
                    x_test_2 = x_1 - 5*bounds
                    flux_sigma_bounds_2 = flux[x_test_1: x_test_2]

                if emline == FeVII_3586 or emline == NeV_3346 or emline == NeV_3426:
                    x_test_1 = x_0 - 4*bounds - 5
                    x_test_2 = x_0 - 5
                    flux_sigma_bounds_2 = flux[x_test_1: x_test_2]

                    if x_test_1 < 0 and emline != NeV_3426:
                        x_test_1 = x_1 + 5
                        x_test_2 = x_1_9 + 5
                        flux_sigma_bounds_2 = flux[x_test_1: x_test_2]
                        
                    if x_test_1 < 0 and emline == NeV_3426:
                        x_test_1 = 0 + 5
                        x_test_2 = x_0 - 5
                        flux_sigma_bounds_2 = flux[x_test_1: x_test_2]
                        
                if emline == NeV_3426 and iau == 'J073623.13+392617.7':
                    flux_sigma_bounds_1 = flux_sigma_bounds_2

                elif emline == NeV_3426 and iau != 'J073623.13+392617.7':
                    flux_sigma_bounds_1 = flux[x_1_9 + 10: x_1 + 8*bounds + 10]
                else:
                    flux_sigma_bounds_1 = flux[x_1 + 5: x_1_9 + 5]

                f_std = np.std(flux_sigma_bounds_1)
                f_std_2 = np.std(flux_sigma_bounds_2)
                flux_sigma = 5*(f_std)
                flux_sigma_2 = 5*(f_std_2)
                flux_sigma_abs_1 = 1*(f_std)

                def gaus(x,amp,mu,sigma, m, c):
                    return amp*np.exp(-(x-mu)**2/(2*sigma**2)) + m*x + c #Defining Gaussian function

                def double_gaussian(x,amp_ems,mu1,sigma1,m1,c1,amp2,mu2,sigma2,m2,c2):
                    return gaus(x,amp_ems,mu1,sigma1,m1,c1) + gaus(x,amp2,mu2,sigma2,m2,c2)

                wavelength_emission = np.argmax(flux_bounds)
                amp_ems = flux_bounds[wavelength_emission]
                wavelength_absorption = np.argmin(flux_bounds)
                amp_abs = flux_bounds[wavelength_absorption]
                amp_abs_1 = -(avg_flux - amp_abs)
                sigma_abs_1 = avg_flux - flux_sigma_abs_1
                sigma_abs_5 = avg_flux - flux_sigma
                sigma_ems = avg_flux + flux_sigma

                o1_wavelength_total = np.abs(wavelength_bounds - OI_2) 
                o1_wavelength_min_total = np.argmin(o1_wavelength_total)
                amp_o1 = flux_bounds[o1_wavelength_min_total]
                mu_o1 = wavelength_bounds[o1_wavelength_min_total]
                sigma_guess_o1 = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_o1)**2)/np.sum(flux_bounds))

                mu_abs = wavelength_bounds[wavelength_absorption]
                mu_ems = wavelength_bounds[wavelength_emission]
                mu = np.sum(wavelength_bounds*flux_bounds)/np.sum(flux_bounds)

                sigma_guess_abs = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_abs)**2)/np.sum(flux_bounds))
                sigma_guess_ems = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_ems)**2)/np.sum(flux_bounds))
                sigma_guess = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu)**2)/np.sum(flux_bounds))
                amp = flux_bounds[wavelength_bounds_min_total]

                if amp_abs < sigma_abs_5:
                    print ('5 Sigma Absorption Line Error' + '\n')
                    continue

                if amp_abs < sigma_abs_1 and amp_ems < sigma_ems:

                    try:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                   p0=[amp_abs_1, mu, sigma_guess, fit[1], 
                                       fit[0]])
                    except RuntimeError:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                   p0=[amp_abs_1, mu, sigma_guess, 1, 
                                       1])

                    print (emline_name + ' Absorption Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))

                    if popt[0] < 0:
                        print ('Absorption Line Error' '\n')
                        continue

                    else:
                        try:
                            popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                       p0=[amp_ems, mu, sigma_guess, fit[1], 
                                           fit[0]])
                        except RuntimeError:
                            popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                       p0=[amp_ems, mu, sigma_guess, 1, 
                                           1])

                        try:
                            perr = np.sqrt(np.diag(pcov))
                        except RuntimeWarning:
                            perr = np.zeros([len(popt)])
                            print ('RuntimeWarning')

                        residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata
                        chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                        reduced_chi_squared = chi_squared / (len(popt) - 1)

                        I = np.sum(gaus(wavelength_bounds, *popt))
                        x_bar = (1/I)*np.sum(wavelength_bounds*gaus(wavelength_bounds, *popt))
                        try:
                            std_1 = np.sqrt((1/I)*np.sum((wavelength_bounds - x_bar)**2*gaus(wavelength_bounds, *popt)))
                            S = (1/(I*std_1**3))*np.sum((wavelength_bounds - x_bar)**3*gaus(wavelength_bounds, *popt))
                        except RuntimeWarning:
                            std_1 = 0    
                            S = 0
                else:
                    try:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,  
                                   p0=[amp_ems, mu, sigma_guess, fit[1], 
                                       fit[0]])
                    except RuntimeError:
                        print ('RuntimeError ')
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,
                                               p0=[amp_ems, mu, sigma_guess, 1,1])
                    try:
                        perr = np.sqrt(np.diag(pcov))
                    except RuntimeWarning:
                        perr = np.zeros([len(popt)])
                        print ('RuntimeWarning')

                    residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata
                    chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                    reduced_chi_squared = chi_squared / (len(popt) - 1)
                    
                    
                    I = np.sum(gaus(wavelength_bounds, *popt))
                    x_bar = (1/I)*np.sum(wavelength_bounds*gaus(wavelength_bounds, *popt))
                    try:
                        std_1 = np.sqrt((1/I)*np.sum((wavelength_bounds - x_bar)**2*gaus(wavelength_bounds, *popt)))
                        S = (1/(I*std_1**3))*np.sum((wavelength_bounds - x_bar)**3*gaus(wavelength_bounds, *popt))
                    except RuntimeWarning:
                        std_1 = 0    
                        S = 0
                    
                    #print ('skewness lit = ' + str(S))
                    #print ('skewness scipy = ' + str(skew(gaus(wavelength_bounds, *popt))))

                if emline == FeX_6374 and popt[1] < 6369: 
                    print ('FeX Error' + '\n')
                    continue

                velocity_meas = c*((popt[1] - emline)/emline)
                velocity_sys = c*tbdata['z'][ind][0]
                velocity_off = velocity_sys - velocity_meas
                sigma = c*(abs(popt[2])/emline)
                fwhm = sigma*2.355

                if emline == FeVII_3586 or emline == FeVII_3760 or emline == FeVII_6086:
                    ip = 99.10  
                if emline == FeX_6374:
                    ip = 233.60
                if emline == NeV_3346 or emline == NeV_3426:
                    ip = 97.12
                    
                if (emline == NeV_3346 or emline == NeV_3426) and len(fitsName) == 83:
                        print (iau + ' (' + fitsName[39:48] + ') ' + emline_name + ' ' +
                            '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                                  +
                            '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma) + 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                        plate = fitsName[39:43]
                        ifu = fitsName[44:48]

                elif (emline == NeV_3346 or emline == NeV_3426) and len(fitsName) == 84 and fitsName[39] != '1':
                    print (iau + ' (' + fitsName[39:49] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[39:43]
                    ifu = fitsName[44:49]

                elif (emline == NeV_3346 or emline == NeV_3426) and len(fitsName) == 84 and fitsName[39] == '1':
                    print (iau + ' (' + fitsName[39:49] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[39:44]
                    ifu = fitsName[45:49]

                elif (emline == NeV_3346 or emline == NeV_3426) and len(fitsName) == 85:
                    print (iau + ' (' + fitsName[39:50] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[39:44]
                    ifu = fitsName[45:50]

                elif emline == FeX_6374 and len(fitsName) == 80:   
                    print (iau + ' ('+ fitsName[36:45] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[36:40]
                    ifu = fitsName[41:45]

                elif emline == FeX_6374 and len(fitsName) == 81 and fitsName[36] != '1':   
                    print (iau + ' ('+ fitsName[36:46] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n') 
                    plate = fitsName[36:40]
                    ifu = fitsName[41:46]

                elif emline == FeX_6374 and len(fitsName) == 81 and fitsName[36] == '1':   
                    print (iau + ' ('+ fitsName[36:46] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n') 
                    plate = fitsName[36:41]
                    ifu = fitsName[42:46]

                elif emline == FeX_6374 and len(fitsName) == 82:
                    print (iau + ' ('+ fitsName[36:47] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[36:41]
                    ifu = fitsName[42:47]

                elif (emline == FeVII_3586 or emline == FeVII_3760 or emline == FeVII_6086) and len(fitsName) == 82:
                    print (iau + ' ('+ fitsName[38:47] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[38:42]
                    ifu = fitsName[43:47]

                elif (emline == FeVII_3586 or emline == FeVII_3760 or emline == FeVII_6086) and len(fitsName) == 83 and fitsName[38] != '1':
                    print (iau + ' (' + fitsName[38:48] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[38:42]
                    ifu = fitsName[43:48]

                elif (emline == FeVII_3586 or emline == FeVII_3760 or emline == FeVII_6086) and len(fitsName) == 83 and fitsName[38] == '1':
                    print (iau + ' (' + fitsName[38:48] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[38:43]
                    ifu = fitsName[44:48]

                elif (emline == FeVII_3586 or emline == FeVII_3760 or emline == FeVII_6086) and len(fitsName) == 84:
                    print (iau + ' (' + fitsName[38:49] + ') ' + emline_name + ' ' +
                        '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(flux_sigma_2)
                              +
                        '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(flux_sigma)+ 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                    plate = fitsName[38:43]
                    ifu = fitsName[44:49]

                hdulist_maps = fits.open('output/fits/%s/' %emline_name + 'maps/' + 'manga-' 
                 + '%s' %plate + '-' + '%s' %ifu + '-MAPS-VOR10-MILESHC-MILESHC.fits')

                em_gau = hdulist_maps[29].data
                
#                 for i in range (1, int(shape)):
#                     OIII_flux = em_gau[14,cp - i:cp + i,pd - i:pd + i]
#                     o,w = OIII_flux.shape 
#                     for j in range (1,o-1):
#                         OIII_flux[j][1:o-1] = 0

                # sigma_threshold = ((2000/2.355)/c)*(wavelength[wavelength_min])
                if sigma > 0 and sigma < 450 and popt[0] > flux_sigma and popt[0] > flux_sigma_2 and mu-15 < popt[1] < mu+15 and popt[0] > 3*perr[0]:

                    newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/5_sigma/%s/'%emline_name + '%s/'%iau
                    if not os.path.exists(newpath):
                        os.makedirs(newpath)
                    f = open(newpath + '%s' %u + '%s' %v + 'data.csv',"w+")
                    f.write('SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + '5 Sigma Threshold (ergs/cm^2/A/spaxel)'
                             + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
                            + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
                            + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
                            + 'Gaussian Sigma (A)' + ',' 
                            + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM' + ',' + 'Measured Velocity (km/s)' 
                            + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
                             + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)'+ ',' 
                            + 'Skewness' + ',' + 'Spaxel Size (pc)' + ',' + 'Galaxy Type' +
                            '\n' + str(iau) + ',' + str(tbdata['z'][ind][0]) + ',' + str(distance) + ',' + str(flux_sigma) + ',' 
                            + str(popt[0])+ ',' + str(perr[0]) 
                            + ','  + str(popt[1]) + ',' + str(perr[1]) +  ','  + str(popt[2]) + ',' + str(perr[2]) + ',' 
                            + str(sigma) + ','  + str(fwhm) + ',' + str(velocity_meas)+ ',' + str(smbh_mass) + ',' + str(sersic_mass) 
                            + ','  + str(velocity_dispersion) + ',' + str(sfr) + ',' + str(chi_squared) + ',' + str(reduced_chi_squared) 
                            + ',' + str(ip) + ',' + str(S) + ',' + str(spaxel_size) + ',' + str(galaxy_type))
                    f.close()

                    fig = plt.figure(figsize=(15,10))
                    ax1 = plt.axes()  
                    ax2 = plt.axes([0.18, 0.7, 0.15, 0.15])
                    ax1.plot(wavelength_bounds_total, flux_correction_total, 'b', label = 'Data')

                    ax2.plot(wavelength_bounds, residuals, 'k')
                    ax2.set_title('Residuals')
                    ax2.set_ylabel('Flux')
                    ax2.set_xlabel(r'Wavelength ($\AA$)')

                    if len(fitsName) == 67:
                        ax1.set_title(iau + ' ('+ fitsName[23:32] + ')')
                    elif len(fitsName) == 68:
                        ax1.set_title(iau + ' ('+ fitsName[23:33] + ')')
                    elif len(fitsName) == 69:
                        ax1.set_title(iau + ' ('+ fitsName[23:34] + ')')

                    ax1.set_ylabel('Flux', fontsize = 15)
                    ax1.set_xlabel(r'Wavelength ($\AA$)', fontsize = 15)
                    ax1.set_title(str(iau) + ' (' + str(plate) + '-' + str(ifu) + ')' )
                    r = wavelength_bounds_total.shape
                    ax1.set_xlim(wavelength_bounds_total[1],wavelength_bounds_total[r[0] - 1])

                    marker = np.argmin(flux_correction_total)
                    marker_min = np.argmax(flux_correction_total)
                    marker_threshold = -(0.035*flux_correction_total[marker_min])


                    if emline == NeV_3426:
                        ax1.text(wavelength[x_1_9 + 20], marker_threshold, 'Neighboring Continuum', fontsize=9)
                    else: 
                        ax1.text(wavelength[x_1 + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)

                    if x_test_1 == 5:
                        ax1.text(wavelength[x_test_1 + 5], marker_threshold, 'NC', fontsize=9)
                    else:
                        ax1.text(wavelength[x_test_1 + 10], marker_threshold, 'Neighboring Continuum', fontsize=9)

                    if emline == FeVII_3586:
                        ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                        ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oii1],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oii1-1], marker_threshold, 'OII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oii2],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                        #ax1.text(wavelength[wavelength_min_oii2-3], marker_threshold, fontsize=9)

                    elif emline == FeVII_3760:
                        ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                        ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oii1],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oii1-1], marker_threshold, 'OII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oii2],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                        #ax1.text(wavelength[wavelength_min_oii2-3], marker_threshold, fontsize=9)
                        ax1.plot(wavelength[wavelength_min_hei],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                        ax1.text(wavelength[wavelength_min_hei-3], marker_threshold, 'OII', fontsize=9)


                    elif emline == FeVII_6086:
                        ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                        ax1.text(wavelength[wavelength_min-4], marker_threshold, 'FeVII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_na],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                        ax1.text(wavelength[wavelength_min_na-2], marker_threshold, 'Na', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oi0],flux_correction_total[marker], marker='|', color = 'red', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oi0-2], marker_threshold, 'OI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_hei1],flux_correction_total[marker], marker='|', color = 'blue', linestyle='None')
                        ax1.text(wavelength[wavelength_min_hei1-2], marker_threshold, 'HeI', fontsize=9)

                    elif emline == FeX_6374:
                        ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                        ax1.text(wavelength[wavelength_min-3], marker_threshold, 'FeX', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oi2],flux_correction_total[marker], marker='|', color = 'purple', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oi2-2], marker_threshold, 'OI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oi1],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oi1-2], marker_threshold, 'OI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_ni],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                        ax1.text(wavelength[wavelength_min_ni-2], marker_threshold, 'NI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_nii1],flux_correction_total[marker], marker='|', color = 'magenta', linestyle='None')
                        ax1.text(wavelength[wavelength_min_nii1-3], marker_threshold, 'NII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_nii2],flux_correction_total[marker], marker='|', color = 'cyan', linestyle='None')
                        ax1.text(wavelength[wavelength_min_nii2-3], marker_threshold, 'NII', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_halpha],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                        ax1.text(wavelength[wavelength_min_halpha-6], marker_threshold, 'Halpha', fontsize=9)

        #                             if emline == OIII_4:
        #                                 plt.plot(wavelength[wavelength_min_oiii3],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
        #                                 plt.text(wavelength[wavelength_min_oiii3-4], marker_threshold, 'OIII', fontsize=9)

                    elif emline == NeV_3346:
                        ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                        ax1.text(wavelength[wavelength_min-3], marker_threshold, 'NeV', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_nevi],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                        ax1.text(wavelength[wavelength_min_nevi-3], marker_threshold, 'NeV', fontsize=9)

                    elif emline == NeV_3426:
                        ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                        ax1.text(wavelength[wavelength_min-3], marker_threshold, 'NeV', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_nev],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                        ax1.text(wavelength[wavelength_min_nev-3], marker_threshold, 'NeV', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_oiii0],flux_correction_total[marker], marker='|', color = 'red', linestyle='None')
                        ax1.text(wavelength[wavelength_min_oiii0-2], marker_threshold, 'OI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_ni0],flux_correction_total[marker], marker='|', color = 'blue', linestyle='None')
                        ax1.text(wavelength[wavelength_min_ni0-2], marker_threshold, 'NI', fontsize=9)
                        ax1.plot(wavelength[wavelength_min_he0],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                        ax1.text(wavelength[wavelength_min_he0-2], marker_threshold, 'He', fontsize=9)


                    ax1.axvline(x = wavelength[x_0], linestyle = '--', color = 'r')
                    ax1.axvline(x = wavelength[x_1], linestyle = '--', color = 'r')
                    ax1.axvline(x = wavelength[x_test_1], linestyle = '-', color = 'k')
                    ax1.axvline(x = wavelength[x_test_2], linestyle = '-', color = 'k')

                    if emline == NeV_3426 and iau != 'J073623.13+392617.7':
                        ax1.axvline(x = wavelength[x_1_9 + 10], linestyle = '-', color = 'k')
                        ax1.axvline(x = wavelength[x_1 + 8*bounds + 10], linestyle = '-', color = 'k')
                    elif emline == NeV_3426 and iau == 'J073623.13+392617.7':
                        print ('J073623.13+392617.7')
                    else:
                        ax1.axvline(x = wavelength[x_1 + 5], linestyle = '-', color = 'k')
                        ax1.axvline(x = wavelength[x_1_9 + 5], linestyle = '-', color = 'k')


                    p_i = wavelength_bounds.shape[0]
                    wavelength_range = np.arange(wavelength_bounds[0],wavelength_bounds[p_i-1], 0.25)

                    if len(popt) == 10:
                        ax1.plot(wavelength_range,double_gaussian(wavelength_range, *popt), 'r-', label = 'Gaussian')
                    else:
                        ax1.plot(wavelength_range,gaus(wavelength_range, *popt), 'r-', label = 'Gaussian')

                    ax1.legend(loc = 1)

                    try:
                        fig.savefig(newpath + iau + '_' + emline_name + '_%s' %u + '%s' %v + '.png', format='png', bbox_inches='tight', dpi=400, pad_inches = 0.1)
                    except ValueError:
                        fig.savefig(newpath + iau + '_' + emline_name + '_%s' %u + '%s' %v + '.png', format='png', dpi=400, pad_inches = 0.1)
                    plt.close()

                    #Emission Line Flux Maps
                    flux_map_bounds_1, flux_map_bounds_2 = popt[1] - 3*popt[2], popt[1] + 3*popt[2]
                    wavelength_map_absolute_1 = np.abs(wavelength - flux_map_bounds_1)
                    wavelength_map_absolute_2 = np.abs(wavelength - flux_map_bounds_2)
                    wavelength_map_min_1 = np.argmin(wavelength_map_absolute_1)
                    wavelength_map_min_2 = np.argmin(wavelength_map_absolute_2)


                    flux_map = flux_nuclear_dimensions[wavelength_map_min_1:wavelength_map_min_2, :, :].sum(axis = (0))

                    f, (ax1) = plt.subplots(1, sharex=False, sharey=False, figsize=(20,8))
                    cax1 = ax1.imshow(flux_map, cmap='gist_heat', origin = 'lower')
                    ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
                    ax1.set_xlabel('Spaxels')
                    ax1.set_ylabel('Spaxels')
                    f.colorbar(cax1, ax = ax1, label = 'Flux')
                    f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps_%s' %u + '%s' %v + '.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
                    #plt.show()
                    plt.close()

    #                     f, (ax1) = plt.subplots(1, sharex=False, sharey=False, figsize=(20,8))
    #                     cax1 = ax1.imshow(flux_map, cmap='gist_heat', origin = 'lower')
    #                     ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
    #                     ax1.set_xlabel('Spaxels')
    #                     ax1.set_ylabel('Spaxels')
    #                     f.colorbar(cax1, ax = ax1, label = 'Flux')
    #                     f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps_%s' %u + '%s' %v + '.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
    #                     plt.show()
    #                     plt.close()

    #                     f, (ax1) = plt.subplots(1, sharex=False, sharey=False, figsize=(20,8))
    #                     cax1 = ax1.imshow(OIII_flux, cmap='gist_heat', origin = 'lower')
    #                     ax1.set_title(iau  + ' ' + emline_name +' OIII Galaxy Flux Map')
    #                     ax1.set_xlabel('Spaxels')
    #                     ax1.set_ylabel('Spaxels')
    #                     f.colorbar(cax1, ax = ax1, label = 'Flux')
    #                     f.savefig(newpath +  iau + '_' + 'OIII_flux_map_%s' %u + '%s' %v + '.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
    #                     #plt.show()
    #                     plt.close()

        hdulist.close()
        drpall.close()               

    return;


# Calling the function 
# emission(emline = FeVII_3760, 
#          bounds = 16, 
#          emline_name = 'FeVII_3760A')
# emission(emline = NeV_3346, 
#          bounds = 16,  
#          emline_name = 'NeV_3346.79A')
# emission(emline = NeV_3426, 
#          bounds = 16,  
#          emline_name = 'NeV_3426.85A')
# emission(emline = FeVII_3586, 
#          bounds = 18, 
#          emline_name = 'FeVII_3586A')
# emission(emline = FeX_6374, 
#          bounds = 16, 
#          emline_name = 'FeX_6374A')
# emission(emline = FeVII_6086, 
#          bounds = 16, 
#          emline_name = 'FeVII_6086A')



In [ ]:
#Importing necessary fits files

def gaus(x,amp,mu,sigma, m, c):
                    return amp*np.exp(-(x-mu)**2/(2*sigma**2)) + m*x + c #Defining Gaussian function

def data_retrieval(emline, emline_name, em_short, redshift_limit, bounds,
                   line_1, line_2, line_3, line_4, line_5, line_6, 
                   n1, n2, n3, n4, n5, n6, 
                   k0, k1, k2, k3, k4, k5, k6,
                   ncl1, ncl2,
                   ip_0):

    for fitsName in glob.glob('output/fits/%s/' %emline_name + 'logcube/' + '*.fits'): 
        hdulist = fits.open(fitsName) 
        hdu = fits.open('zoo2MainSpecz.fits')
        drpall = fits.open('drpall-v2_5_3.fits') 
        dapall = fits.open('dapall-v2_5_3-2.3.0.fits') 
        
        tbdata = drpall[1].data 
        ind = np.where(tbdata['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
        iau = tbdata['nsa_iauname'][ind][0] #Finding SDSS Name
        redshift = tbdata['z'][ind][0]
        
        plateifu1 = tbdata['plateifu'][ind][0]
        if len(plateifu1) == 9:
            plate = plateifu1[0:4]
            ifu = plateifu1[5:9]
        if len(plateifu1) == 10 and plateifu1[0] != '1':
            plate = plateifu1[0:4]
            ifu = plateifu1[5:10]
        if len(plateifu1) == 10 and plateifu1[0] == '1':
            plate = plateifu1[0:5]
            ifu = plateifu1[6:10]
        if len(plateifu1) == 11:
            plate = plateifu1[0:5]
            ifu = plateifu1[6:11]
            
        print (str(iau) + ' ' + str(emline_name) + ' (' + str(plate) + '-' + str(ifu) + ')')
            
        hdulist_maps = fits.open('output/fits/%s/' %emline_name + 'maps/' + 'manga-' 
             + '%s' %plate + '-' + '%s' %ifu + '-MAPS-VOR10-MILESHC-MILESHC.fits')
        oiii_gau = hdulist_maps[29].data
            
        tbdata_1 = dapall[1].data 
        ind_1 = np.where(tbdata_1['mangaid'] == str(hdulist[0].header[59])) #Finding MaNGA ID
    
        galaxy_zoo = hdu[1].data
        ra = str(iau[1:3]) + ':' + str(iau[3:5] + ':' + str(iau[5:7]) + iau[7:10]) 
        dec = str(iau[10:13]) + ':' + str(iau[13:15] + ':' + str(iau[15:17]) + iau[17:19]) 
        ind_2 = np.where(galaxy_zoo['rastring'] == ra) #Finding MaNGA ID
        galaxy_type = galaxy_zoo['gz2class'][ind_2]
        
        wavelength_import = hdulist[4].data #Importing Wavelength Values
        wavelength = wavelength_import/(tbdata['z'][ind][0] + 1)
        flux_import = hdulist[1].data
        
        velocity_dispersion = tbdata_1['STELLAR_SIGMA_1RE'][ind][0] 
        smbh_mass = (3.1*(velocity_dispersion/200)**4)*(10**8)
        sersic_mass = tbdata['nsa_sersic_mass'][ind][0] #Stellar mass from K-correction fit in h-2 solar masses to sersic magnitudes.
        distance = ((tbdata['z'][ind][0])*c)/H_0
        spaxel_size = (2*math.tan(0.25)/206265)*(distance*10**6)
        sfr = tbdata_1['SFR_1RE'][ind][0] 
        ip = ip_0
        
        if (emline == NeV_3346 or emline == NeV_3426 or emline == FeVII_3586) and redshift < redshift_limit:
            hdulist.close()
            drpall.close()  
            continue
        
        spaxe1,spaxel2,spaxel3 = flux_import.shape
        center = int(spaxel2/2)
        print ('Total Array Size = ' + str(spaxel2) + ' x ' + str(spaxel3) + '\n')
        
        for l in range (spaxel2-1):
            for m in range (spaxel3-1):
                print ('Current Pixel = ' + str(l) + ' x ' + str(m))
                flux = flux_import[:,l,m]

#         for j in range (spaxel1):
#             for k in range (1,spaxel3 - 1):
#                 flux_nuclear_dimensions[j][k][1:spaxel3 - 1] = 0
#                 OIII_flux[14][k][1:spaxel3 - 1] = 0
#                 spaxel1, spaxel2, spaxel3 = flux_nuclear_dimensions.shape
#                 print ('Current Array Size = ' + str(spaxel2) + ' x ' +  str(spaxel3) + '\n')
                
#                 flux = flux_nuclear_dimensions.sum(axis = (1, 2))
#                 flux = flux_nuclear_dimensions.sum(axis = (0))

                wavelength_absolute = np.abs(wavelength - emline)
                wavelength_min = np.argmin(wavelength_absolute)
                wav_neigh_1 = np.abs(wavelength - line_1)
                wav_neigh_2 = np.abs(wavelength - line_2)
                wav_neigh_3 = np.abs(wavelength - line_3)
                wav_neigh_4 = np.abs(wavelength - line_4)
                wav_neigh_5 = np.abs(wavelength - line_5)
                wav_neigh_6 = np.abs(wavelength - line_6)
                wavemin_1 = np.argmin(wav_neigh_1)
                wavemin_2 = np.argmin(wav_neigh_2)
                wavemin_3 = np.argmin(wav_neigh_3)
                wavemin_4 = np.argmin(wav_neigh_4)
                wavemin_5 = np.argmin(wav_neigh_5)
                wavemin_6 = np.argmin(wav_neigh_6)

                #Spectrum parameters
                x_0, x_1 = wavelength_min - bounds, wavelength_min + bounds #Setting lower and upper bound for linear fit
                if x_0 < 0:
                    x_0 = 0
                lb, ub = x_0 - 10*bounds, x_1 + 10*bounds #Setting lower and upper bound for linear fit
                if lb < 0:
                    lb = 0
                wavelength_bounds_total, flux_bounds_total = wavelength[lb:ub], flux[lb:ub]

                wavelength_total = np.abs(wavelength_bounds_total - emline) 
                try: 
                    wavelength_min_total = np.argmin(wavelength_total)
                except ValueError:
                    print ('Empty Array' + '\n')
                    continue

                wavelength_mask_total = np.ma.masked_array(wavelength_bounds_total,
                                                (wavelength_bounds_total != wavelength_bounds_total[0])& 
                                                (wavelength_bounds_total != wavelength_bounds_total[-1]))
                wavelength_compressed_total = wavelength_mask_total.compressed() 

                flux_mask_total = np.ma.masked_array(flux_bounds_total, 
                                            (flux_bounds_total != flux_bounds_total[0])& 
                                            (flux_bounds_total != flux_bounds_total[-1])) 
                flux_compressed_total = flux_mask_total.compressed()

                if flux_compressed_total.shape[0] > 2:
                    flux_1,flux_2 = flux_compressed_total[0],flux_compressed_total[-1]
                    flux_compressed_total = [flux_1, flux_2]

                if flux_compressed_total[0] == 0:
                    print ('No Spectrum Available' + '\n')
                    continue

                else:
                    polyfit_total  = np.polyfit(wavelength_compressed_total,flux_compressed_total,1)
                    fit_total = np.poly1d(polyfit_total)
                    flux_linear_total = fit_total[1]*wavelength_bounds_total + fit_total[0] 
                    flux_correction_total = flux_bounds_total - flux_linear_total 

                for i in flux_correction_total:
                    if i < 0:
                        neg_i = np.argmin(flux_correction_total)
                        flux_correction_total = flux_correction_total + abs(flux_correction_total[neg_i])
                        break

                q_0, q_1 = wavelength_min_total - bounds, wavelength_min_total + bounds
                if q_0 < 0:
                    q_0 = 0
                wavelength_bounds, flux_bounds = wavelength_bounds_total[q_0:q_1], flux_correction_total[q_0:q_1]

                wavelength_bounds_em = np.abs(wavelength_bounds - emline) 
                wavelength_bounds_min = np.argmin(wavelength_bounds_em)

                wavelength_mask  = np.ma.masked_array(wavelength_bounds,
                                                 (wavelength_bounds != wavelength_bounds[0])& 
                                                 (wavelength_bounds != wavelength_bounds[-1]))
                wavelength_compressed = wavelength_mask.compressed() 

                flux_mask = np.ma.masked_array(flux_bounds, (flux_bounds != flux_bounds[0]) 
                                       & (flux_bounds != flux_bounds[-1]))
                flux_compressed = flux_mask.compressed() 

                if flux_compressed.shape[0] > 2:
                    flux_1,flux_2 = flux_compressed[0],flux_compressed[-1]
                    flux_compressed = [flux_1, flux_2]

                polyfit = np.polyfit(wavelength_compressed,flux_compressed,1) #Fitting the line to the data
                fit = np.poly1d(polyfit)
                flux_linear = fit[1]*wavelength_bounds + fit[0] #Determining linear fit parameters

                ncr = x_1 + 4*bounds + 5
                ncl_1 = x_0 - ncl1*bounds - 5
                ncl_2 = x_0 - ncl2*bounds - 5
                ncl_bounds = flux[ncl_1: ncl_2]
                
                if ncl_1 < 0 and emline != NeV_3426:
                    ncl_1 = x_1 + 5 #Setting left bounds equal to default right bounds
                    ncl_2 = ncr 
                    ncl_bounds = flux[ncl_1: ncl_2]

                if ncl_1 < 0 and emline == NeV_3426:
                    ncl_1 = 5
                    ncl_2 = x_0 - 5
                    ncl_bounds = flux[ncl_1: ncl_2]
                    
                if emline == NeV_3426 and iau == 'J073623.13+392617.7':
                    ncr_bounds = ncl_bounds
                elif emline == NeV_3426 and iau != 'J073623.13+392617.7':
                    ncr_bounds = flux[ncr + 5: x_1 + 8*bounds + 10]
                else:
                    ncr_bounds = flux[x_1 + 5: ncr]
                    
                ncr_std, ncl_std = np.std(ncr_bounds), np.std(ncl_bounds)
                ncr_sigma, ncl_sigma = 5*(ncr_std), 5*(ncl_std)
                abs_sigma = 1*(ncr_std)
                
                emission, absorption = np.argmax(flux_bounds), np.argmin(flux_bounds)
                avg_flux = np.average(flux_linear)
                abs_sigma_1 = avg_flux - abs_sigma
                abs_sigma_5 = avg_flux - ncr_sigma
                ems_sigma_1 = avg_flux + ncr_sigma
                
                mu = np.sum(wavelength_bounds*flux_bounds)/np.sum(flux_bounds)
                mu_abs, mu_ems = wavelength_bounds[absorption],wavelength_bounds[emission]
                
                sigma_guess = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu)**2)/np.sum(flux_bounds))
                sigma_guess_ems = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_ems)**2)/np.sum(flux_bounds))
                sigma_guess_abs = np.sqrt(np.sum(flux_bounds*(wavelength_bounds-mu_abs)**2)/np.sum(flux_bounds))
                
                amp = flux_bounds[wavelength_bounds_min]
                amp_ems, amp_abs = flux_bounds[emission], flux_bounds[absorption]
                amp_abs_1 = -(avg_flux - amp_abs)

                if amp_abs < abs_sigma_5:
                    print ('5 Sigma Absorption Line Error' + '\n')
                    continue
                    
                try:
                    popt_1,pcov_1 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp_ems,mu,sigma_guess,fit[1],fit[0]])
                except RuntimeError:
                    popt_1,pcov_1 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp_ems,mu,sigma_guess,1,1])
                try:
                    perr_1 = np.sqrt(np.diag(pcov_1))
                except RuntimeWarning:
                    perr_1 = np.zeros([len(popt_1)])
                    print ('RuntimeWarning')
                    
                try:
                    popt_2,pcov_2 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp,mu,sigma_guess,fit[1],fit[0]])
                except RuntimeError:
                    popt_2,pcov_2 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp,mu,sigma_guess,1,1])
                try:
                    perr_2 = np.sqrt(np.diag(pcov_2))
                except RuntimeWarning:
                    perr_2 = np.zeros([len(popt_2)])
                    print ('RuntimeWarning')
                    
                try:
                    popt_3,pcov_3 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp_ems,mu_ems,sigma_guess,fit[1],fit[0]])
                except RuntimeError:
                    popt_3,pcov_3 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp_ems,mu_ems,sigma_guess,1,1])
                try:
                    perr_3 = np.sqrt(np.diag(pcov_3))
                except RuntimeWarning:
                    perr_3 = np.zeros([len(popt_3)])
                    print ('RuntimeWarning')
                    
                try:
                    popt_4,pcov_4 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp,mu_ems,sigma_guess,fit[1],fit[0]])
                except RuntimeError:
                    popt_4,pcov_4 = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp,mu_ems,sigma_guess,1,1])
                try:
                    perr_4 = np.sqrt(np.diag(pcov_4))
                except RuntimeWarning:
                    perr_4 = np.zeros([len(popt_4)])
                    print ('RuntimeWarning')
                    
                test = [perr_1[0],perr_2[0],perr_3[0],perr_4[0]]

                best_fit = np.argmin(test)

                if best_fit == 0:
                    popt = popt_1
                    perr = perr_1
                if best_fit == 1:
                    popt = popt_2
                    perr = perr_2
                if best_fit == 2:
                    popt = popt_3
                    perr = perr_3
                if best_fit == 3:
                    popt = popt_4
                    perr = perr_4

                residuals = flux_bounds - gaus(wavelength_bounds, *popt) #Determines the uncertainty in ydata
                chi_squared =  np.sum(((residuals)** 2)/(gaus(wavelength_bounds, *popt)))
                reduced_chi_squared = chi_squared / (len(popt) - 1)

                I = np.sum(gaus(wavelength_bounds, *popt))
                x_bar = (1/I)*np.sum(wavelength_bounds*gaus(wavelength_bounds, *popt))
                std_1 = np.sqrt((1/I)*np.sum((wavelength_bounds - x_bar)**2*gaus(wavelength_bounds, *popt)))
                skew_lit = (1/(I*std_1**3))*np.sum((wavelength_bounds - x_bar)**3*gaus(wavelength_bounds, *popt))
                skew_scipy = skew(gaus(wavelength_bounds, *popt))

                if amp_abs < abs_sigma_1 and amp_ems < ems_sigma_1:
                    try:
                        popt, pcov = curve_fit(gaus,wavelength_bounds, flux_bounds,p0=[amp_abs_1,mu,sigma_guess,fit[1],fit[0]])
                    except RuntimeError:
                        popt, pcov = curve_fit(gaus,wavelength_bounds,flux_bounds,p0=[amp_abs_1,mu,sigma_guess,1,1])
                        
                    print (emline_name + ' Absorption Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]))

                    if popt[0] < 0:
                        print ('Absorption Line Error' '\n')
                        continue

                if emline == FeX_6374 and popt[1] < 6369: 
                    print ('FeX Error' + '\n')
                    continue

                velocity_meas = c*((popt[1] - emline)/emline)
                velocity_sys = c*tbdata['z'][ind][0]
                velocity_off = velocity_sys - velocity_meas
                sigma = c*(abs(popt[2])/emline)
                fwhm = sigma*2.355

                print (iau + ' (' + str(plate) + '-' + str(ifu) + ') ' + emline_name +
                            '\n' + 'Five Sigma Left (ergs/cm^2/A/spaxel) = ' + str(ncl_sigma)
                                  +
                            '\n' + 'Five Sigma Right (ergs/cm^2/A/spaxel) = ' + str(ncr_sigma) + 
                              '\n'  + emline_name + ' Flux (ergs/cm^2/A/spaxel) = ' + str(popt[0]) + '\n')
                
                if sigma > 0 and sigma < 450 and popt[0] > ncr_sigma and popt[0] > ncl_sigma and mu-15 < popt[1] < mu+15 and popt[0] > 3*perr[0]:
                    newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/5_sigma/%s/'%emline_name + '%s/'%iau
                    if not os.path.exists(newpath):
                        os.makedirs(newpath)
                    f = open(newpath + '%s' %l + '%s' %m + 'data.csv',"w+")
                    f.write('SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
                            + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
                            + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
                            + 'Gaussian Sigma (A)' + ',' 
                            + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM' + ',' + 'Measured Velocity (km/s)' 
                            + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
                             + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)'+ ',' 
                            + 'Skewness' + ',' + 'Spaxel Size (pc)' + ',' + 'Galaxy Type' +
                            '\n' + str(iau) + ',' + str(tbdata['z'][ind][0]) + ',' + str(distance) + ','  
                            + str(popt[0])+ ',' + str(perr[0]) 
                            + ','  + str(popt[1]) + ',' + str(perr[1]) +  ','  + str(popt[2]) + ',' + str(perr[2]) + ',' 
                            + str(sigma) + ','  + str(fwhm) + ',' + str(velocity_meas)+ ',' + str(smbh_mass) + ',' + str(sersic_mass) 
                            + ','  + str(velocity_dispersion) + ',' + str(sfr) + ',' + str(chi_squared) + ',' + str(reduced_chi_squared) 
                            + ',' + str(ip) + ',' + str(skew_lit) + ',' + str(spaxel_size) + ',' + str(galaxy_type))
                    f.close()

                    marker = np.argmin(flux_correction_total)
                    marker_min = np.argmax(flux_correction_total)
                    marker_threshold = -(0.035*flux_correction_total[marker_min])
                    p_i = wavelength_bounds.shape[0]
                    wavelength_range = np.arange(wavelength_bounds[0],wavelength_bounds[p_i-1], 0.25)
                    r = wavelength_bounds_total.shape
                    
                    fig = plt.figure(figsize=(15,10))
                    ax1 = plt.axes()  
                    ax2 = plt.axes([0.18, 0.7, 0.15, 0.15])
                    ax1.plot(wavelength_bounds_total, flux_correction_total, 'b', label = 'Data')
                    ax2.plot(wavelength_bounds, residuals, 'k')
                    ax2.set_title('Residuals')
                    ax2.set_ylabel('Flux')
                    ax2.set_xlabel(r'Wavelength ($\AA$)')
                    ax1.set_title(iau + ' ('+ str(plate) + '-' + str(ifu) + ')')
                    ax1.set_ylabel('Flux', fontsize = 15)
                    ax1.set_xlabel(r'Wavelength ($\AA$)', fontsize = 15)
                    ax1.set_title(str(iau) + ' (' + str(plate) + '-' + str(ifu) + ')' )
                    ax1.set_xlim(wavelength_bounds_total[1],wavelength_bounds_total[r[0] - 1])
                    ax1.plot(wavelength[wavelength_min],flux_correction_total[marker], marker='|', color = 'black', linestyle='None')
                    ax1.text(wavelength[wavelength_min-k0], marker_threshold, em_short, fontsize=9)
                    ax1.plot(wavelength[wavemin_1],flux_correction_total[marker], marker='|', color = 'purple', linestyle='None')
                    ax1.text(wavelength[wavemin_1-k1], marker_threshold, n1, fontsize=9)
                    ax1.plot(wavelength[wavemin_2],flux_correction_total[marker], marker='|', color = 'orange', linestyle='None')
                    ax1.text(wavelength[wavemin_2-k2], marker_threshold, n2, fontsize=9)
                    ax1.plot(wavelength[wavemin_3],flux_correction_total[marker], marker='|', color = 'green', linestyle='None')
                    ax1.text(wavelength[wavemin_3-k3], marker_threshold, n3, fontsize=9)
                    ax1.plot(wavelength[wavemin_4],flux_correction_total[marker], marker='|', color = 'magenta', linestyle='None')
                    ax1.text(wavelength[wavemin_4-k4], marker_threshold, n4, fontsize=9)
                    ax1.plot(wavelength[wavemin_5],flux_correction_total[marker], marker='|', color = 'cyan', linestyle='None')
                    ax1.text(wavelength[wavemin_5-k5], marker_threshold, n5, fontsize=9)
                    ax1.plot(wavelength[wavemin_6],flux_correction_total[marker], marker='|', color = 'pink', linestyle='None')
                    ax1.text(wavelength[wavemin_6-k6], marker_threshold, n6, fontsize=9)
                    ax1.axvline(x = wavelength[x_0], linestyle = '--', color = 'r')
                    ax1.axvline(x = wavelength[x_1], linestyle = '--', color = 'r')
                    ax1.axvline(x = wavelength[ncl_1], linestyle = '-', color = 'k')
                    ax1.axvline(x = wavelength[ncl_2], linestyle = '-', color = 'k')
                    if ncl_1 == 5:
                        ax1.text(wavelength[ncl_1 + 5], marker_threshold, 'NC', fontsize=9)
                    elif emline == NeV_3346 and (ncl_bounds[0] == ncr_bounds[0]):
                        print ('Same Bounds')
                    else:
                        ax1.text(wavelength[ncl_1 + 10], marker_threshold, 'Neighboring Continuum', fontsize=9)
                    if emline == NeV_3426 and iau != 'J073623.13+392617.7':
                        ax1.axvline(x = wavelength[ncr + 5], linestyle = '-', color = 'k')
                        ax1.axvline(x = wavelength[x_1 + 8*bounds + 10], linestyle = '-', color = 'k')
                        ax1.text(wavelength[ncr + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)
                    elif emline == NeV_3426 and iau == 'J073623.13+392617.7':
                        print ('J073623.13+392617.7')
                    else:
                        ax1.axvline(x = wavelength[x_1 + 5], linestyle = '-', color = 'k')
                        ax1.axvline(x = wavelength[ncr], linestyle = '-', color = 'k')
                        ax1.text(wavelength[x_1 + 15], marker_threshold, 'Neighboring Continuum', fontsize=9)
                    ax1.plot(wavelength_range,gaus(wavelength_range, *popt), 'r-', label = 'Gaussian')
                    ax1.legend(loc = 1)
                    try:
                        fig.savefig(newpath + iau + '_' + emline_name + '_%s' %l + '%s' %m + '.png', format='png', bbox_inches='tight', dpi=400, pad_inches = 0.1)
                    except ValueError:
                        fig.savefig(newpath + iau + '_' + emline_name + '_%s' %l + '%s' %m + '.png', format='png', dpi=400, pad_inches = 0.1)
                    plt.close()
    hdulist.close()
    drpall.close()               
    return;
# data_retrieval (emline = NeV_3346, emline_name = 'NeV_3346.79A', em_short = 'NeV', redshift_limit = 0.0882, bounds = 15, 
#                 line_1 = NeV_3426, line_2 = 0, line_3 = 0, line_4 = 0, line_5 = 0, line_6 = 0,
#                 n1 = 'NeV', n2 = '', n3 = '', n4 = '', n5 = '', n6 = '', 
#                 k0 = 3, k1 = 3, k2 = 0, k3 = 0, k4 = 0, k5 = 0, k6 = 0, 
#                 ncl1 = 4, ncl2 = 0,
#                 ip_0 = 97.12)
# data_retrieval (emline = NeV_3426, emline_name = 'NeV_3426.85A', em_short = 'NeV', redshift_limit = 0.0628, bounds = 15,
#                 line_1 = NeV_3346, line_2 = OIII_0, line_3 = NI_0, line_4 = He_0, line_5 = 0, line_6 = 0, 
#                 n1 = 'NeV', n2 = 'OIII', n3 = 'NI', n4 = 'He', n5 = '', n6 = '', 
#                 k0 = 3, k1 = 2, k2 = 4, k3 = 2, k4 = 2, k5 = 0, k6 = 0, 
#                 ncl1 = 4, ncl2 = 0,
#                 ip_0 = 97.12)
# data_retrieval (emline = FeVII_3586, emline_name = 'FeVII_3586A', em_short = 'FeVII', redshift_limit = 0.0156, bounds = 15,
#                 line_1 = 0, line_2 = 0, line_3 = 0, line_4 = 0, line_5 = 0, line_6 = 0, 
#                 n1 = '', n2 = '', n3 = '', n4 = '', n5 = '', n6 = '', 
#                 k0 = 5, k1 = 0, k2 = 0, k3 = 0, k4 = 0, k5 = 0, k6 = 0, 
#                 ncl1 = 4, ncl2 = 0,
#                 ip_0 = 99.10)
# data_retrieval (emline = FeVII_3760, emline_name = 'FeVII_3760A', em_short = 'FeVII', redshift_limit = 0, bounds = 15,  
#                 line_1 = OII_1, line_2 = OII_2, line_3 = He_I, line_4 = 0, line_5 = 0, line_6 = 0, 
#                 n1 = 'OII', n2 = ' ', n3 = 'He', n4 = '', n5 = '', n6 = '', 
#                 k0 = 5, k1 = 1, k2 = 0, k3 = 2, k4 = 0, k5 = 0, k6 = 0, 
#                 ncl1 = 7, ncl2 = 3,
#                 ip_0 = 99.10)
# data_retrieval (emline = FeVII_6086, emline_name = 'FeVII_6086A', em_short = 'FeVII', redshift_limit = 0, bounds = 15, 
#                 line_1 = Na, line_2 = OI_0, line_3 = He_I_1, line_4 = 0, line_5 = 0, line_6 = 0, 
#                 n1 = 'Na', n2 = 'OI', n3 = 'He', n4 = '', n5 = '', n6 = '', 
#                 k0 = 5, k1 = 2, k2 = 2, k3 = 2, k4 = 0, k5 = 0, k6 = 0, 
#                 ncl1 = 7, ncl2 = 3,
#                 ip_0 = 99.10) 
data_retrieval (emline = FeX_6374, emline_name = 'FeX_6374A', em_short = 'FeX', redshift_limit = 0, bounds = 15,  
                line_1 = OI_1, line_2 = OI_2, line_3 = NI,  line_4 = NII_1, line_5 = NII_2, line_6 = H_Alpha, 
                n1 = 'OI', n2 = 'OI', n3 = 'NI', n4 = 'NII', n5 = 'NII', n6 = '', 
                k0 = 3, k1 = 2, k2 = 2, k3 = 2, k4 = 3, k5 = 3, k6 = 6, 
                ncl1 = 7, ncl2 = 3,
                ip_0 = 233.60)

J030051.56+004826.5 FeX_6374A (9192-12702)
Total Array Size = 72 x 72

Current Pixel = 0 x 0
No Spectrum Available

Current Pixel = 0 x 1
No Spectrum Available

Current Pixel = 0 x 2
No Spectrum Available

Current Pixel = 0 x 3
No Spectrum Available

Current Pixel = 0 x 4
No Spectrum Available

Current Pixel = 0 x 5
No Spectrum Available

Current Pixel = 0 x 6
No Spectrum Available

Current Pixel = 0 x 7
No Spectrum Available

Current Pixel = 0 x 8
No Spectrum Available

Current Pixel = 0 x 9
No Spectrum Available

Current Pixel = 0 x 10
No Spectrum Available

Current Pixel = 0 x 11
No Spectrum Available

Current Pixel = 0 x 12
No Spectrum Available

Current Pixel = 0 x 13
No Spectrum Available

Current Pixel = 0 x 14
No Spectrum Available

Current Pixel = 0 x 15
No Spectrum Available

Current Pixel = 0 x 16
No Spectrum Available

Current Pixel = 0 x 17
No Spectrum Available

Current Pixel = 0 x 18
No Spectrum Available

Current Pixel = 0 x 19
No Spectrum Available

Current Pixel = 0 x

No Spectrum Available

Current Pixel = 4 x 49
No Spectrum Available

Current Pixel = 4 x 50
No Spectrum Available

Current Pixel = 4 x 51
No Spectrum Available

Current Pixel = 4 x 52
No Spectrum Available

Current Pixel = 4 x 53
No Spectrum Available

Current Pixel = 4 x 54
No Spectrum Available

Current Pixel = 4 x 55
No Spectrum Available

Current Pixel = 4 x 56
No Spectrum Available

Current Pixel = 4 x 57
No Spectrum Available

Current Pixel = 4 x 58
No Spectrum Available

Current Pixel = 4 x 59
No Spectrum Available

Current Pixel = 4 x 60
No Spectrum Available

Current Pixel = 4 x 61
No Spectrum Available

Current Pixel = 4 x 62
No Spectrum Available

Current Pixel = 4 x 63
No Spectrum Available

Current Pixel = 4 x 64
No Spectrum Available

Current Pixel = 4 x 65
No Spectrum Available

Current Pixel = 4 x 66
No Spectrum Available

Current Pixel = 4 x 67
No Spectrum Available

Current Pixel = 4 x 68
No Spectrum Available

Current Pixel = 4 x 69
No Spectrum Available

Current Pix

FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 8 x 45
No Spectrum Available

Current Pixel = 8 x 46
No Spectrum Available

Current Pixel = 8 x 47
No Spectrum Available

Current Pixel = 8 x 48
No Spectrum Available

Current Pixel = 8 x 49
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 8 x 50
No Spectrum Available

Current Pixel = 8 x 51
No Spectrum Available

Current Pixel = 8 x 52
No Spectrum Available

Current Pixel = 8 x 53
No Spectrum Available

Current Pixel = 8 x 54
No Spectrum Available

Current Pixel = 8 x 55
No Spectrum Available

Current Pixel = 8 x 56
No Spectrum Available

Current Pixel = 8 x 57
No Spectrum Available

Current Pixel = 8 x 58
No Spectrum Available

Current Pixel = 8 x 59
No Spectrum Available

Current Pixel = 8 x 60
No Spectrum Available

Current Pixel = 8 x 61
No Spectrum Available

Current Pixel = 8 x 62
No Spectrum Available

Current

FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 10 x 41
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 10 x 42
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 10 x 43
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 10 x 44
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 10 x 45
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 10 x 46
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 10 x 47
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.007505547125905705
Absorption Line Error

Current Pixel = 10 x 48
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.00750554

No Spectrum Available

Current Pixel = 12 x 4
No Spectrum Available

Current Pixel = 12 x 5
No Spectrum Available

Current Pixel = 12 x 6
No Spectrum Available

Current Pixel = 12 x 7
No Spectrum Available

Current Pixel = 12 x 8
No Spectrum Available

Current Pixel = 12 x 9
No Spectrum Available

Current Pixel = 12 x 10
No Spectrum Available

Current Pixel = 12 x 11
No Spectrum Available

Current Pixel = 12 x 12
No Spectrum Available

Current Pixel = 12 x 13
No Spectrum Available

Current Pixel = 12 x 14
No Spectrum Available

Current Pixel = 12 x 15
No Spectrum Available

Current Pixel = 12 x 16
No Spectrum Available

Current Pixel = 12 x 17
No Spectrum Available

Current Pixel = 12 x 18
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 12 x 19
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 12 x 20
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.00619844627

FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 34
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 35
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 36
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 37
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 38
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 39
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 40
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Absorption Line Error

Current Pixel = 13 x 41
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.2274223414545507
Abs

No Spectrum Available

Current Pixel = 14 x 59
No Spectrum Available

Current Pixel = 14 x 60
No Spectrum Available

Current Pixel = 14 x 61
No Spectrum Available

Current Pixel = 14 x 62
No Spectrum Available

Current Pixel = 14 x 63
No Spectrum Available

Current Pixel = 14 x 64
No Spectrum Available

Current Pixel = 14 x 65
No Spectrum Available

Current Pixel = 14 x 66
No Spectrum Available

Current Pixel = 14 x 67
No Spectrum Available

Current Pixel = 14 x 68
No Spectrum Available

Current Pixel = 14 x 69
No Spectrum Available

Current Pixel = 14 x 70
No Spectrum Available

Current Pixel = 15 x 0
No Spectrum Available

Current Pixel = 15 x 1
No Spectrum Available

Current Pixel = 15 x 2
No Spectrum Available

Current Pixel = 15 x 3
No Spectrum Available

Current Pixel = 15 x 4
No Spectrum Available

Current Pixel = 15 x 5
No Spectrum Available

Current Pixel = 15 x 6
No Spectrum Available

Current Pixel = 15 x 7
No Spectrum Available

Current Pixel = 15 x 8
No Spectrum Available


FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 24
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 25
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 26
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 27
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 28
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 29
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 30
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) = -0.0061984462769253765
Absorption Line Error

Current Pixel = 16 x 31
FeX_6374A Absorption Flux (ergs/cm^2/A/spaxel) =

In [131]:
#creating 'results' file

directory = '5_sigma'

f= open('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + 'results' + ".csv","a+")

    
f.write('FeVII_3586' + ',' + 'FeVII_3760' + ',' +'FeVII_6086' + ',' +'FeX_6374' + ',' +'NeV_3346.79' + ',' +'NeV_3426.85' + ',' + 'Spaxel Array'
                                + ',' + 'SDSS Name' + ',' + 'Redshift' + ',' + 'Distance (Mpc)' + ',' + '5 Sigma Threshold (ergs/cm^2/A/spaxel)'
                                 + ',' + 'Gaussian Amplitude (ergs/cm^2/A/spaxel)' + ',' 
                                + 'Gaussian Amplitude Error (ergs/cm^2/A/spaxel)'+ ',' 
                                + 'Gaussian Centroid (A)' + ',' + 'Gaussian Centroid Error (A)' + ',' 
                                + 'Gaussian Sigma (A)' + ',' 
                                + 'Gaussian Sigma Error (A)'+ ','+ 'Sigma (km/s)' + ',' + 'FWHM (km/s)' + ',' + 'Measured Velocity (km/s)' 
                                + ',' + 'SMBH Mass(Solar Masses)' + ',' + 'Sersic Mass (Solar Masses)'+  ',' + 'Velocity Dispersion (km/s)' + ','
                                 + 'SFR (h-2 Msun/yr)' + ',' + 'Chi Squared' + ',' + 'Reduced Chi Squared' +  ',' + 'Ionization Energy (eV)' +  ','
                                + 'Skewness' + ',' + 'Spaxel Size (pc)' + ',' + 'Galaxy Type')
f.close()

In [128]:
#importing data into 'results' file.  

directory = '5_sigma'

def output_file(emline):
    path = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + '%s/' %emline)
    z = 0
    for i in path:
        for j in range (2, 75):
            if path[z][0] == 'J':
                g = ('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + '%s/'%emline + '%s/'%path[z] 
                     + '%i' %j + '%i' %j + 'data.csv' )
                try:
                    data = pd.read_csv(g, delimiter = ',')
                except FileNotFoundError:
                    continue
                sdss = data['SDSS Name']
                redshift = data['Redshift']
                d = data['Distance (Mpc)']
                flux_threshold = data['5 Sigma Threshold (ergs/cm^2/A/spaxel)']
                amp = data['Gaussian Amplitude (ergs/cm^2/A/spaxel)']
                amp_err = data['Gaussian Amplitude Error (ergs/cm^2/A/spaxel)']
                cen = data['Gaussian Centroid (A)']
                cen_err = data['Gaussian Centroid Error (A)']
                gau_sig = data['Gaussian Sigma (A)']
                gau_sig_err = data['Gaussian Sigma Error (A)']
                sig = data['Sigma (km/s)']
                fwhm = data['FWHM']
                v = data['Measured Velocity (km/s)'] 
                smbh_mass = data['SMBH Mass(Solar Masses)']
                sersic_mass = data['Sersic Mass (Solar Masses)']
                v_disp = data['Velocity Dispersion (km/s)']
                sfr = data['SFR (h-2 Msun/yr)']
                chi = data['Chi Squared']
                r_chi = data['Reduced Chi Squared']
                ip = data['Ionization Energy (eV)'] 
                skew = data['Skewness']
                spaxel_size = data['Spaxel Size (pc)']
                galaxy_type = data['Galaxy Type'] 

                f= open('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + 'results' + ".csv","a+")

                if emline == 'FeVII_3586A':
                    f.write('\n' + '1' + ',' + '' + ',' + '' + ',' + '' + ',' + '' + ',' + '' + ','  + str(j) + str (j) + ',' + str(sdss[0]) +  ',' + str(redshift[0]) + ','+ str(d[0]) +  ',' 
                            + str(flux_threshold[0]) + ',' + str(amp[0]) + ','+ str(amp_err[0]) + ',' + str(cen[0])+ ',' 
                            + str(cen_err[0])+ ',' + str(gau_sig[0])+ ',' + str(gau_sig_err[0])+ ',' + str(sig[0]) + ',' + str(fwhm[0])
                            + ',' + str(v[0])+ ',' + str(smbh_mass[0])+ ',' + str(sersic_mass[0])+ ',' + str(v_disp[0])+ ',' + str(sfr[0]) + ',' + str(chi[0]) + ',' + str(r_chi[0])
                            + ',' + str(ip[0]) + ',' + str(skew[0]) + ',' + str(spaxel_size[0]) + ',' + str(galaxy_type[0]))
                if emline == 'FeVII_3760A':
                    f.write('\n' + '' + ',' + '1' + ',' + '' + ',' + '' + ',' + '' + ',' + '' + ',' + str(j) + str (j) + ',' + str(sdss[0]) +  ',' + str(redshift[0]) + ','+ str(d[0]) +  ',' 
                            + str(flux_threshold[0]) + ',' + str(amp[0]) + ','+ str(amp_err[0]) + ',' + str(cen[0])+ ',' 
                            + str(cen_err[0])+ ',' + str(gau_sig[0])+ ',' + str(gau_sig_err[0])+ ',' + str(sig[0]) + ',' + str(fwhm[0])
                            + ',' + str(v[0])+ ',' + str(smbh_mass[0])+ ',' + str(sersic_mass[0])+ ',' + str(v_disp[0])+ ',' + str(sfr[0]) + ',' + str(chi[0]) + ',' + str(r_chi[0])
                            + ',' + str(ip[0]) + ',' + str(skew[0]) + ',' + str(spaxel_size[0]) + ',' + str(galaxy_type[0]))
                if emline == 'FeVII_6086A':
                    f.write('\n' + '' + ',' + '' + ',' + '1' + ',' + '' + ',' + '' + ',' + '' + ',' + str(j) + str (j) + ',' + str(sdss[0]) +  ',' + str(redshift[0]) + ','+ str(d[0]) +  ',' 
                            + str(flux_threshold[0]) + ',' + str(amp[0]) + ','+ str(amp_err[0]) + ',' + str(cen[0])+ ',' 
                            + str(cen_err[0])+ ',' + str(gau_sig[0])+ ',' + str(gau_sig_err[0])+ ',' + str(sig[0]) + ',' + str(fwhm[0])
                            + ',' + str(v[0])+ ',' + str(smbh_mass[0])+ ',' + str(sersic_mass[0])+ ',' + str(v_disp[0])+ ',' + str(sfr[0]) + ',' + str(chi[0]) + ',' + str(r_chi[0])
                            + ',' + str(ip[0]) + ',' + str(skew[0]) + ',' + str(spaxel_size[0]) + ',' + str(galaxy_type[0]))
                if emline == 'FeX_6374A':
                    f.write('\n' + '' + ',' + '' + ',' + '' + ',' + '1' + ',' + '' + ',' + '' + ',' + str(j) + str (j) + ',' + str(sdss[0]) +  ',' + str(redshift[0]) + ','+ str(d[0]) +  ',' 
                            + str(flux_threshold[0]) + ',' + str(amp[0]) + ','+ str(amp_err[0]) + ',' + str(cen[0])+ ',' 
                            + str(cen_err[0])+ ',' + str(gau_sig[0])+ ',' + str(gau_sig_err[0])+ ',' + str(sig[0]) + ',' + str(fwhm[0])
                            + ',' + str(v[0])+ ',' + str(smbh_mass[0])+ ',' + str(sersic_mass[0])+ ',' + str(v_disp[0])+ ',' + str(sfr[0]) + ',' + str(chi[0]) + ',' + str(r_chi[0])
                            + ',' + str(ip[0]) + ',' + str(skew[0]) + ',' + str(spaxel_size[0]) + ',' + str(galaxy_type[0]))
                if emline == 'NeV_3346.79A':
                    f.write('\n' + '' + ',' + '' + ',' + '' + ',' + '' + ',' + '1' + ',' + '' + ',' + str(j) + str (j) + ',' + str(sdss[0]) +  ',' + str(redshift[0]) + ','+ str(d[0]) +  ',' 
                            + str(flux_threshold[0]) + ',' + str(amp[0]) + ','+ str(amp_err[0]) + ',' + str(cen[0])+ ',' 
                            + str(cen_err[0])+ ',' + str(gau_sig[0])+ ',' + str(gau_sig_err[0])+ ',' + str(sig[0]) + ',' + str(fwhm[0])
                            + ',' + str(v[0])+ ',' + str(smbh_mass[0])+ ',' + str(sersic_mass[0])+ ',' + str(v_disp[0])+ ',' + str(sfr[0]) + ',' + str(chi[0]) + ',' + str(r_chi[0])
                            + ',' + str(ip[0]) + ',' + str(skew[0]) + ',' + str(spaxel_size[0]) + ',' + str(galaxy_type[0]))
                if emline == 'NeV_3426.85A':
                    f.write('\n' + '' + ',' + '' + ',' + '' + ',' + '' + ',' + '' + ',' + '1' + ',' + str(j) + str (j) + ',' + str(sdss[0]) +  ',' + str(redshift[0]) + ','+ str(d[0]) +  ',' 
                            + str(flux_threshold[0]) + ',' + str(amp[0]) + ','+ str(amp_err[0]) + ',' + str(cen[0])+ ',' 
                            + str(cen_err[0])+ ',' + str(gau_sig[0])+ ',' + str(gau_sig_err[0])+ ',' + str(sig[0]) + ',' + str(fwhm[0])
                            + ',' + str(v[0])+ ',' + str(smbh_mass[0])+ ',' + str(sersic_mass[0])+ ',' + str(v_disp[0])+ ',' + str(sfr[0]) + ',' + str(chi[0]) + ',' + str(r_chi[0])
                            + ',' + str(ip[0]) + ',' + str(skew[0]) + ',' + str(spaxel_size[0]) + ',' + str(galaxy_type[0]))
                f.close()
        z += 1
    
    return;
output_file(emline = 'FeVII_3586A')
output_file(emline = 'FeVII_3760A')
output_file(emline = 'FeVII_6086A')
output_file(emline = 'FeX_6374A')
output_file(emline = 'NeV_3346.79A')
output_file(emline = 'NeV_3426.85A')

#sorting data by SDSS name

df = pd.read_csv('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + 'results.csv')
sorted_data = df.sort_values("SDSS Name")
sorted_data.to_csv('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + 'results.csv', index=False)

In [101]:
#Determining Coronal Line distance

drpall = fits.open('drpall-v2_5_3.fits') #Opening drpall file
tbdata = drpall[1].data #Accessing drpall data

directory = '5_sigma'

f= open('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + 'clr' + ".csv","a+")
f.write('Emission Lines' + ',' + 'clr_min' + ',' + 'clr_max' + ',' + 'clr')

def output_file(emline):
    path = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + '%s/' %emline )
    z = 0
    for i in path:
        if path[z][0] == 'J':
            g = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + '%s/'%emline + '%s/'%path[z])
            array_size = []
            for j in g:
                if j[-3:] == 'csv' and len(j) == 10:
                    array_size.append(int(j[0:1]))
                if j[-3:] == 'csv' and len(j) == 12:
                    array_size.append(int(j[0:2]))
            
            ind = np.where(tbdata['nsa_iauname'] == str(path[z]))
            distance = ((tbdata['z'][ind][0])*c)/H_0
            spaxel_size = (2*math.tan(0.25)/206265)*(distance*10**6)
            
            high_pix = array_size[np.argmax(array_size)]*spaxel_size
            low_pix = array_size[np.argmin(array_size)]*spaxel_size
            clr = (high_pix - low_pix)
            if clr == 0:
                clr = 1
            f.write('\n' + str(emline) + ',' + str(low_pix) + ',' + str(high_pix) + ',' + str(clr))
        z += 1
    
    return;
output_file(emline = 'FeVII_3586A')
output_file(emline = 'FeVII_3760A')
output_file(emline = 'FeVII_6086A')
output_file(emline = 'FeX_6374A')
output_file(emline = 'NeV_3346.79A')
output_file(emline = 'NeV_3426.85A')

f.close()

In [3]:
#creating LaTeX table 

directory = '5_sigma_galaxy'

data_path = ('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + 'results.csv' )
data = pd.read_csv(data_path, delimiter = ',')
fevii_3586 = data['FeVII_3586']
fevii_3760 = data['FeVII_3760']
fevii_6086 = data['FeVII_6086']
fex = data['FeX_6374']
nev = data['NeV_3346.79']
nev_1 = data['NeV_3346.79']
sdss = data['SDSS Name']
redshift = data['Redshift']
d = data['Distance (Mpc)']
flux_threshold = data['5 Sigma Threshold (ergs/cm^2/A/spaxel)']
amp = data['Gaussian Amplitude (ergs/cm^2/A/spaxel)']
amp_err = data['Gaussian Amplitude Error (ergs/cm^2/A/spaxel)']
cen = data['Gaussian Centroid (A)']
cen_err = data['Gaussian Centroid Error (A)']
gau_sig = data['Gaussian Sigma (A)']
gau_sig_err = data['Gaussian Sigma Error (A)']
sig = data['Sigma (km/s)']
fwhm = data['FWHM (km/s)']
v = data['Measured Velocity (km/s)'] 
smbh_mass = data['SMBH Mass(Solar Masses)']
sersic_mass = data['Sersic Mass (Solar Masses)']
v_disp = data['Velocity Dispersion (km/s)']
sfr = data['SFR (h-2 Msun/yr)']
chi = data['Chi Squared']
r_chi = data['Reduced Chi Squared']
ip = data['Ionization Energy (eV)']

i = 0
for i in range(1,len(sdss)-1):
    i += 1
    if fevii_3586[i] == 1:
        detection = '[FeVII]$\lambda$3586'
    if fevii_3760[i] == 1:
        detection = '[FeVII]$\lambda$3760'
    if fevii_6086[i] == 1:
        detection = '[FeVII]$\lambda$6086'
    if fex[i] == 1:
        detection = '[FeX]$\lambda6374$'
    if nev[i] == 1:
        detection = '[NeV]$\lambda$3347'
    if nev_1[i] == 1:
        detection = '[NeV]$\lambda$3426'

    print (str(sdss[i]) + ' & ' + str(round_sig(redshift[i], 2)) + ' & ' + str(round_sig(d[i],4)) + ' & ' 
           + str(round_sig(amp[i],4)) + r'$\pm$' + str(round_sig(amp_err[i],2))
           + ' & ' + str(round_sig(cen[i],4)) + r'$\pm$' + str(round_sig(cen_err[i],1)) + ' & ' + str(round_sig(gau_sig[i],2)) + r'$\pm$' 
           + str(round_sig(gau_sig_err[i],2))
           + ' & ' + str(round_sig(fwhm[i],4)) + ' & ' + str(round_sig(smbh_mass[i],9)) + ' & ' + str(round_sig(v_disp[i],3)) + ' & ' 
           + str(round_sig(sfr[i],2)) +  ' & ' + str(detection) + r' \\')

In [46]:
#removing duplicate files from separate directories

directory = '5_sigma_galaxy_skewness'
directory_1 = '5_sigma_galaxy_2'

def duplicate_removal(emline):
    path = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + '%s/' %emline)
    path_1 = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/%s/' %directory_1 + '%s/' %emline)
    for i in path_1:
        path_2 = '/Users/jamesnegus/Google_Drive_CU/primary_code/%s/' %directory_1 + '%s/' %emline + '%s/' %i
        for j in path:
            if i == j:
                shutil.rmtree(path_2)
                
    return;
# duplicate_removal(emline = 'FeVII_3586A')
# duplicate_removal(emline = 'FeVII_3760A')
# duplicate_removal(emline = 'FeVII_6086A')
# duplicate_removal(emline = 'FeX_6374A')
# duplicate_removal(emline = 'NeV_3346.79A')

In [3]:
#retrieval of fits files from MaNGA servers

drpall = fits.open('drpall-v2_5_3.fits')
tbdata = drpall[1].data 

directory = '5_sigma_galaxy_2'

def fits_retrieval(emline):
    path = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/%s/' %directory + '%s/' %emline)
#     path = os.listdir('/Users/jamesnegus/Google_Drive_CU/primary_code/output/%s/' %directory + '%s/' %emline + 'logcube/')
    z = 0
    for i in path:
        if path[z][0] == 'J':
            ind = np.where(tbdata['nsa_iauname'] == path[z])
            manga_id = tbdata['plateifu'][ind][0] 
            if len(manga_id) == 9:
                plate = manga_id[0:4]
                ifu = manga_id[5:10]
            if len(manga_id) == 10 and manga_id[0] != '1':
                plate = manga_id[0:4]
                ifu = manga_id[5:10]
            if len(manga_id) == 10 and manga_id[0] == '1':
                plate = manga_id[0:5]
                ifu = manga_id[6:10]
            if len(manga_id) == 11:
                plate = manga_id[0:5]
                ifu = manga_id[6:11]
                
#         if path[z][0] == 'm':
#             if len(path[z]) == 50:
#                     plate = path[z][6:10]
#                     ifu = path[z][11:15]

#             if len(path[z]) == 51 and path[z][6] != '1':
#                 plate = path[z][6:10]
#                 ifu = path[z][11:16]

#             if len(path[z]) == 51 and path[z][6] == '1':
#                 plate = path[z][6:11]
#                 ifu = path[z][12:16]

#             if len(path[z]) == 52:
#                 plate = path[z][6:11]
#                 ifu = path[z][12:17]
            newpath = r'/Users/jamesnegus/Google_Drive_CU/primary_code/output/fits2/%s/'%emline + 'logcube'
            if not os.path.exists(newpath):
                os.makedirs(newpath)
            os.system('rsync -avz --password-file=rsync_pass rsync://sdss@dtn01.sdss.'
                      'utah.edu/sas/mangawork/manga/spectro/analysis/MPL-8/VOR10-MILESHC-MILESHC/'
                      +str(plate)+'/'+str(ifu)+'/manga-'+str(plate)+'-'+str(ifu)
                      +'-LOGCUBE-VOR10-MILESHC-MILESHC.fits.gz output/fits2/%s/' %emline + 'logcube/')
            os.system('gunzip output/fits2/%s/' %emline + 'logcube/' + 'manga-'+str(plate)+'-'+str(ifu)+'-LOGCUBE-VOR10-MILESHC-MILESHC.fits.gz')

            os.system('rsync -avz --password-file=rsync_pass rsync://sdss@dtn01.sdss.'
                      'utah.edu/sas/mangawork/manga/spectro/analysis/MPL-8/VOR10-MILESHC-MILESHC/'
                      +str(plate)+'/'+str(ifu)+'/manga-'+str(plate)+'-'+str(ifu)
                      +'-MAPS-VOR10-MILESHC-MILESHC.fits.gz output/fits2/%s/' %emline + 'maps/')
            os.system('gunzip output/fits2/%s/' %emline + 'maps/' + 'manga-'+str(plate)+'-'+str(ifu)+'-MAPS-VOR10-MILESHC-MILESHC.fits.gz')
        z += 1

    return;
fits_retrieval(emline = 'FeVII_3586A')
# fits_retrieval(emline = 'FeVII_3760A')
fits_retrieval(emline = 'FeVII_6086A')
fits_retrieval(emline = 'FeX_6374A')
# fits_retrieval(emline = 'NeV_3346.79A')
# fits_retrieval(emline = 'NeV_3426.85A')

# for plate in chain(range(7443, 7444, 1), range(7495, 7496, 1), range(7815,7816, 1), 
#                    range(7957,7994, 1), range(8077,8098, 1), range(8131,8159, 1), 
#                    range(8239, 8275, 1), range(8309, 8487, 1),range(8547, 8658, 1), 
#                    range(8711, 8729, 1), range(8931, 9096, 1), range(9181, 9197, 1), 
#                    range(9485, 9515, 1), range(9862, 9895, 1), range(10001, 10002, 1), 
#                    range(10141, 10148, 1), range(10213, 10222, 1), range(10492, 10520, 1)):
# #                         range(9673, 9679, 1),    
        
# for ifu in chain(range(1901,1903, 1), range(3701, 3705, 1), range(6101, 6105, 1), 
#                  range(9101, 9103, 1), range(12701, 12706, 1)):


In [ ]:
#CLR Distance

data_path = ('/Users/jamesnegus/Google Drive/' + 'clr.csv' )
data = pd.read_csv(data_path, delimiter = ',')
emline = data['Emission Lines']
clr_min = data['clr_min']/2000
clr_max = data['clr_max']/2000
clr = data['clr']/2000

x = emline.shape[0]

def histogram (f, t):
    f = []
    for i in range (0, x):
        if emline[i] == t:
            f.append(clr[i])
    print ('Average CLR Distance (kpc) = ' + str(np.average(f)))
    plt.figure(figsize=(20,10))
    plt.hist(f, bins = 'auto')
    plt.title (t + ' (' + str(len(f)) + ' Galaxies)')
    plt.xlabel ('Distance (kpc)')
    plt.show()
    plt.close()
    return;

histogram (f = f1, t = 'FeVII_3586A')
histogram (f = f2, t = 'FeVII_3760A')
histogram (f = f3, t = 'FeVII_6086A')
histogram (f = f4, t = 'FeX_6374A')
histogram (f = f5, t = 'NeV_3346.79A')
histogram (f = f6, t = 'NeV_3426.85A')


In [ ]:
#Flux Maps

#                     f, (ax1) = plt.subplots(1, sharex=False, sharey=False, figsize=(20,8))
#                     cax1 = ax1.imshow(flux_map, cmap='gist_heat', origin = 'lower')
#                     ax1.set_title(iau  + ' ' + emline_name + ' ' + ' Galaxy Flux Map')
#                     ax1.set_xlabel('Spaxels')
#                     ax1.set_ylabel('Spaxels')
#                     f.colorbar(cax1, ax = ax1, label = 'Flux')
#                     f.savefig(newpath +  iau + '_' + 'emission_line_flux_maps_%s' %l + '%s' %m + '.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
#                     #plt.show()
#                     plt.close()

#                     f, (ax1) = plt.subplots(1, sharex=False, sharey=False, figsize=(20,8))
#                     cax1 = ax1.imshow(OIII_flux, cmap='gist_heat', origin = 'lower')
#                     ax1.set_title(iau  + ' ' + emline_name +' OIII Galaxy Flux Map')
#                     ax1.set_xlabel('Spaxels')
#                     ax1.set_ylabel('Spaxels')
#                     f.colorbar(cax1, ax = ax1, label = 'Flux')
#                     f.savefig(newpath +  iau + '_' + 'OIII_flux_map_%s' %u + '%s' %v + '.png', dpi=600, format='png', bbox_inches='tight', pad_inches = 0)
#                     #plt.show()
#                     plt.close()

In [37]:
x = []
def td(x[1], x[2], x[3]):
    for j in range(1,4):
        print (x[j])
    return;
td(x[1] = 'x_1', x[2] = 'x_2', x[3] = 'x_3')

SyntaxError: invalid syntax (<ipython-input-37-6222f7450732>, line 2)